In [182]:
import os.path
import datetime
import pandas as pd
import numpy as np
import hashlib
import matplotlib
import matplotlib.pyplot as plt
from scipy.spatial.distance import squareform, pdist

%matplotlib inline

# Display all columns in Pandas
pd.set_option('display.max_columns', None) 

I aim to produce a challenging but not impossible average pace prediction for each runner. I will first get the runner's non personal best average pace and find the most similar non personal best average pace to that runner. I will then find the personal best average pace of these runners. By average these pb times, or getting the best personal best average pace of these runners, I will be able to produce a predicted average pace. 

In [183]:
# Constants
pace_threshold = 1
#euclidean_distance_threshold = 200

In [184]:
# Read in Split Timings
df = pd.read_csv('../../data/1km_splits_full.csv', encoding = "ISO-8859-1")

In [185]:
# Drop unnecessary columns
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'bib','team', 'rank', 'nationality'], axis = 1)

In [186]:
df.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
0,2017,7efc0cb7e70d7cd9da46a58bbf01af66,Male,Senior Men (23-39),5.281863,NaN,NaN,4.375000,NaN,5.443366,6.483539,9.900000,4.980392,8.205502,4.044218,9.433333,5.237037,5.646341,NaN,NaN,10.649660,6.691441,6.487179,11.881481,5.323333,5.707143,NaN,NaN,8.030973,NaN,NaN,6.102041,NaN,8.604167,NaN,7.747126,NaN,NaN,7.788288,NaN,NaN,9.016667,5.236486,1141.900000,NaN,7.012357
1,2017,1e4ba9a22b3fecb9136ecfcf7f470868,Male,Senior Men (23-39),5.147059,NaN,NaN,4.519737,NaN,5.501618,6.510288,10.100000,4.970588,8.098706,4.047619,9.429167,5.255556,5.743902,NaN,NaN,11.159864,6.887387,6.810897,12.225926,5.376667,5.800000,NaN,NaN,8.781711,NaN,NaN,5.768707,NaN,8.729167,NaN,7.235632,NaN,NaN,7.869369,NaN,NaN,9.300000,5.072072,1156.983333,15.083333,7.097568
2,2017,0d928f7482046900e19dba217d7befa5,Male,Senior Men (23-39),5.544118,NaN,NaN,4.372807,NaN,5.500000,6.981481,10.769697,5.689542,8.812298,4.452381,10.020833,5.629630,6.292683,NaN,NaN,11.411565,6.979730,6.519231,NaN,11.085000,5.870238,NaN,NaN,8.631268,NaN,NaN,5.323129,NaN,9.222222,NaN,7.497126,NaN,NaN,7.918919,NaN,NaN,9.804762,4.772523,1193.000000,51.100000,7.352225
3,2017,93f20063908ba5d48724dcf31740a52a,Male,Senior Men (23-39),5.283088,NaN,NaN,4.379386,NaN,5.441748,6.489712,11.700000,5.741830,9.105178,4.812925,10.220833,5.677778,6.516260,NaN,NaN,12.340136,7.243243,7.243590,13.255556,5.170000,5.839286,NaN,NaN,8.510324,NaN,NaN,6.119048,NaN,9.104167,NaN,7.370690,NaN,NaN,7.585586,NaN,NaN,9.238095,4.808559,1203.650000,61.750000,7.466542
4,2017,7479fb90beffad048c5f772ba1915b4e,Male,Senior Men (23-39),5.145833,NaN,NaN,4.245614,NaN,5.582524,6.615226,9.887879,4.973856,8.249191,3.901361,8.366667,5.162963,6.024390,NaN,NaN,10.870748,7.063063,6.448718,11.896296,5.868333,7.513095,NaN,NaN,12.036873,NaN,NaN,4.996599,NaN,8.180556,NaN,6.422414,NaN,NaN,8.414414,NaN,NaN,10.057143,5.141892,1211.633333,69.733333,7.211069


In [187]:
df.shape

(19579, 46)

In [188]:
df.groupby('category').count()

,Year,name_mask,gender,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
category,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Masters Men 1 (40-49),7881,7881,7881,5994,560,954,5990,801,7749,7435,7379,7389,7116,7098,6886,6834,6869,1697,816,5475,6215,6176,5779,5764,5339,1816,540,2726,1811,534,2722,4254,714,4262,714,3707,524,712,3705,523,711,4408,4931,4930,7877
Masters Men 2 (50-59),3393,3393,3393,2592,220,376,2588,373,3312,3145,3116,3119,2966,2944,2848,2814,2838,675,330,2172,2462,2438,2230,2220,2004,736,183,974,733,183,970,1608,239,1606,239,1403,173,235,1398,172,236,1633,1805,1805,3388
Masters Men 3 (60-69),520,520,520,371,43,61,370,74,502,458,453,454,423,417,388,387,384,70,35,281,309,304,261,254,216,100,23,76,100,23,77,180,13,178,13,148,22,13,148,21,13,162,183,183,520
Masters Men 4 (70-79),29,29,29,23,1,4,23,2,28,27,25,26,23,23,21,21,20,2,1,14,15,15,12,12,7,3,0,2,3,0,2,5,0,4,0,4,0,0,4,0,0,4,4,4,29
Masters Women 1 (40-49),719,719,719,560,51,69,560,81,702,668,660,665,633,625,607,601,602,166,72,466,537,530,474,473,431,140,29,240,140,29,239,341,61,343,61,309,27,61,310,27,61,370,397,397,717
Masters Women 2 (50-59),287,287,287,220,15,37,220,26,278,261,259,258,239,237,227,228,224,51,23,171,191,187,169,168,150,60,12,69,60,12,69,117,20,116,20,100,12,20,101,12,20,121,133,133,285
Masters Women 3 (60-69),26,26,26,21,0,3,21,2,24,21,22,21,18,18,17,16,16,8,5,7,12,12,9,9,8,3,1,4,3,1,4,8,0,8,0,7,1,0,7,1,0,7,8,8,26
Senior Men (23-39),6042,6042,6042,4555,501,700,4555,674,5943,5737,5703,5706,5511,5515,5366,5315,5374,1325,639,4360,4955,4917,4701,4687,4354,1493,367,2247,1481,355,2248,3418,586,3423,586,3046,344,580,3044,344,580,3628,3972,3967,6036
Senior Women (23-39),646,646,646,519,31,69,518,51,630,614,606,607,585,579,561,558,563,156,70,440,498,493,460,457,421,134,33,237,134,33,234,326,66,324,66,284,32,66,285,32,66,348,380,380,645


In [189]:
#df = df[~df["AVG_Pace"] > 12 & df.category == 'Masters Men 3 (60-69)']

#df[(df["AVG_Pace"] <11) & (df.category == 'Masters Men 3 (60-69)')]



In [190]:
df[(df["AVG_Pace"] <12) & (df.category == 'Masters Women 2 (50-59)')]


,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
2471,2017,6edbc123d2c45fa61b34a1f47eae4a2c,Female,Masters Women 2 (50-59),10.267157,NaN,NaN,9.618421,NaN,11.441748,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.442442
2485,2017,dd9e346cc77bdb575a062caf1e045922,Female,Masters Women 2 (50-59),11.279412,NaN,NaN,10.745614,NaN,12.241100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.422042
2494,2017,086c45154e617591ad9887ce3358be2d,Female,Masters Women 2 (50-59),10.833333,NaN,NaN,11.392544,NaN,13.122977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.782952
2595,2016,36da23ac15e20c3b3e35b879229da995,Female,Masters Women 2 (50-59),7.949755,NaN,NaN,5.844298,NaN,8.763754,9.913580,13.748485,7.372549,10.580906,14.908163,13.258333,7.566667,8.939024,18.055556,NaN,14.758503,8.916667,8.650641,15.911111,6.843333,8.625000,NaN,NaN,11.520649,NaN,NaN,8.564626,16.606918,NaN,8.194444,NaN,17.140693,NaN,NaN,14.343434,NaN,NaN,8.497748,1753.933333,433.900000,11.018994
5066,2016,518c744fb20eb7d4c43afd66c1410b57,Female,Masters Women 2 (50-59),10.808824,NaN,NaN,8.978070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.893447
5079,2016,8cb28bf1befd988376fe7152453f08e9,Female,Masters Women 2 (50-59),11.164216,NaN,NaN,11.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.207108
5081,2016,85d4d22c35b487dcb6fa7c4fcfca9ae3,Female,Masters Women 2 (50-59),13.436275,NaN,NaN,9.083333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.259804
7405,2015,cbf15f23ef5ead61ed9491f0e92aa9b8,Female,Masters Women 2 (50-59),9.091912,NaN,NaN,7.708333,NaN,9.885113,14.232510,18.296970,9.911765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.521101
7546,2015,ea16550cfbbfd6cd4327356dc80126e6,Female,Masters Women 2 (50-59),9.000000,NaN,NaN,9.063596,NaN,11.915858,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.993151
7559,2015,7c2c1575d30df2b22d3662dc49bc41bb,Female,Masters Women 2 (50-59),10.167892,NaN,NaN,8.837719,NaN,11.711974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.239195


In [191]:
#df = df.drop([7767])
#df = df.drop([10182])

In [192]:
df.loc[1613]

Year                                                        2017
name_mask                       4db319a671ab6e2b099df28df89f85c4
gender                                                      Male
category                                   Masters Men 1 (40-49)
Chamonix to Delevret                                      8.6826
Chamonix to La Charme                                        NaN
Chamonix to St-Gervais                                       NaN
Delevret to St-Gervais                                   8.11842
La Charme to St-Gervais                                      NaN
St-Gervais to Contamines                                 10.7799
Contamines to La Balme                                   14.7181
La Balme to Bonhomme                                     22.7212
Bonhomme to Chapieux                                     10.8431
Chapieux to Col Seigne                                   17.9725
Col Seigne to Lac Combal                                 10.3231
Lac Combal to Mt-Favre   

In [193]:
# Filter out NaN times
df = df.dropna(subset=['time'])
# Sort by name (first) and year (second)
# Sorting by year, since it might make sense to later only generate prediction for latest non-pb time
df.sort_values(['name_mask', 'Year'], ascending=[True, False], inplace=True)
# Show sample of data
df.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
11228,2013,0001fff82652ce3818504f26771297a3,Male,Senior Men (23-39),7.925245,NaN,NaN,6.611842,NaN,10.315534,10.477366,15.990909,10.611111,15.563107,9.758503,22.025000,11.170370,13.402439,NaN,NaN,33.187075,12.585586,11.990385,23.470370,10.875000,13.505952,21.299663,NaN,NaN,15.997354,NaN,NaN,27.305031,NaN,18.133333,NaN,27.787879,NaN,NaN,18.045455,NaN,NaN,18.114865,2586.933333,1351.983333,16.089557
3623,2016,0002b9e98b5761bb4aa0960616f45ca6,Male,Masters Men 1 (40-49),8.459559,NaN,NaN,7.653509,NaN,11.849515,17.141975,18.469697,12.617647,16.720065,24.044218,17.337500,11.022222,13.654472,66.444444,NaN,18.125850,14.306306,12.666667,23.518519,11.310000,17.688095,NaN,NaN,19.547198,NaN,NaN,14.591837,28.166667,NaN,11.530556,NaN,19.720779,NaN,NaN,14.439394,NaN,NaN,11.335586,2646.166667,1326.133333,17.694491
5634,2015,000bf78327bdfa8c6298fc9a76921305,Male,Masters Men 1 (40-49),7.971814,NaN,NaN,6.657895,NaN,9.449838,12.952675,17.209091,10.323529,14.173139,21.414966,18.554167,10.270370,11.796748,35.777778,23.616667,NaN,12.972973,12.865385,23.362963,11.135000,12.634524,NaN,NaN,17.690265,NaN,NaN,12.363946,20.327044,NaN,12.508333,NaN,17.738095,NaN,NaN,16.020202,NaN,NaN,10.786036,2323.750000,1054.500000,15.222938
7880,2014,000d01e06e9abfe51cdb08ff2567f125,Male,Senior Men (23-39),7.726716,NaN,NaN,6.125000,NaN,8.590615,10.203704,14.993939,7.905229,11.296117,7.071429,15.108333,8.466667,10.317073,NaN,NaN,21.159864,10.364865,10.740385,18.259259,9.588333,11.288095,NaN,NaN,15.786136,NaN,NaN,11.030612,18.198113,NaN,13.405556,NaN,18.824675,NaN,NaN,18.515152,NaN,NaN,13.027027,2021.466667,809.733333,12.416371
15238,2009,00156070bc3ed3fd8b6b87dabd51c2fc,Male,Senior Men (23-39),NaN,NaN,8.779845,NaN,NaN,8.166667,10.508230,15.206061,8.872549,12.690939,7.278912,14.308333,7.914815,8.349593,NaN,NaN,24.034014,10.572072,11.000000,22.844444,8.900000,13.594048,17.599327,NaN,NaN,12.955026,NaN,NaN,18.968553,NaN,13.488889,NaN,20.811688,NaN,NaN,16.530303,NaN,NaN,16.921171,2193.000000,899.700000,13.491108


# Filter out runners who only ran one race

Runner's have ran at least 2 races to be included in the case base, ie. a non personal best time and a personal best time

In [194]:
# Count number of races per runner
v = df.name_mask.value_counts()
# Get runners who ran more than one race
multi_race_runners = v.index[v.gt(1)]
# Filter by multi_race_runners
df = df[df.name_mask.isin(multi_race_runners)]

In [195]:
df.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
8150,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,2312.483333,1100.750000,14.183180
10507,2013,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
13323,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),10.694853,NaN,NaN,10.660088,NaN,11.444984,14.790123,18.772727,10.356209,14.632686,9.476190,17.833333,11.033333,13.252033,NaN,NaN,25.986395,11.813063,16.256410,25.833333,15.586667,17.473810,NaN,21.075426,NaN,NaN,16.559829,NaN,21.588050,NaN,11.688889,NaN,NaN,10.767196,NaN,NaN,11.762452,NaN,NaN,2581.333333,1344.616667,15.188612
12954,2011,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),8.261029,NaN,NaN,7.385965,NaN,9.022654,11.744856,15.790909,8.666667,13.215210,8.047619,16.012500,9.270370,11.483740,NaN,NaN,30.527211,10.997748,12.743590,21.881481,13.003333,16.636905,NaN,21.569343,NaN,NaN,19.384615,NaN,20.207547,NaN,12.561111,NaN,NaN,11.484127,NaN,NaN,10.839080,NaN,NaN,2370.466667,1133.750000,13.945114
18340,2008,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),NaN,4.723913,NaN,NaN,8.472222,9.040453,12.572016,16.218182,9.852941,15.737864,11.806122,21.025000,8.951852,8.869919,NaN,NaN,25.017007,19.596847,13.961538,26.729630,10.395000,14.508333,27.432660,NaN,NaN,21.005291,NaN,NaN,24.845912,NaN,17.430556,NaN,30.506494,NaN,NaN,15.212121,NaN,NaN,18.905405,2707.316667,1450.333333,16.367387


# Merge

In [196]:
# Get Personal Best
pb = df[df.time.groupby(df.name_mask).apply(lambda x: x == x.min())]
# Drop Year and category to avoid duplication on merge
pb = pb.drop(['Year', 'category'], axis = 1)
# Get Non-Personal Best Times
npb = df[~df.time.groupby(df.name_mask).apply(lambda x: x == x.min())]

In [197]:
pb.head()

,name_mask,gender,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
10507,003cc0338235bb1862eb7b9bdd31ec9c,Male,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
12954,006db776bb4048de7f357c632a8778ac,Male,8.261029,NaN,NaN,7.385965,NaN,9.022654,11.744856,15.790909,8.666667,13.215210,8.047619,16.012500,9.270370,11.483740,NaN,NaN,30.527211,10.997748,12.743590,21.881481,13.003333,16.636905,NaN,21.569343,NaN,NaN,19.384615,NaN,20.207547,NaN,12.561111,NaN,NaN,11.484127,NaN,NaN,10.83908,NaN,NaN,2370.466667,1133.750000,13.945114
8431,00a7ade381ebb6d25e1aacb0bc7129e9,Male,8.466912,NaN,NaN,7.383772,NaN,8.750809,12.213992,16.269697,10.627451,14.220065,9.401361,17.579167,10.533333,10.540650,NaN,NaN,29.227891,11.637387,15.320513,23.751852,11.983333,14.132143,NaN,NaN,20.764012,NaN,NaN,15.326531,26.301887,NaN,16.358333,NaN,23.008658,NaN,NaN,18.101010,NaN,NaN,11.027027,2456.833333,1245.100000,15.121991
15237,00b39183bebaffe426e376f7efc8ae1b,Male,NaN,NaN,7.547287,NaN,NaN,9.362460,12.662551,16.833333,11.153595,13.598706,8.969388,17.229167,9.374074,10.439024,NaN,NaN,21.806122,10.963964,12.903846,19.774074,9.646667,11.486905,16.872054,NaN,NaN,15.801587,NaN,NaN,20.562893,NaN,13.461111,NaN,19.062771,NaN,NaN,14.969697,NaN,NaN,12.225225,2192.983333,899.683333,13.769848
6180,011b597a8c4121461b659f50dbd19d5a,Male,8.411765,NaN,NaN,7.717105,NaN,10.508091,12.713992,19.566667,11.323529,15.192557,23.520408,19.091667,11.370370,12.223577,29.333333,27.775,NaN,15.608108,16.535256,24.925926,11.051667,14.361905,NaN,NaN,21.212389,NaN,NaN,13.040816,25.194969,NaN,12.630556,NaN,21.954545,NaN,NaN,17.929293,NaN,NaN,12.461712,2595.100000,1325.850000,16.626208


In [198]:
npb.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
8150,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,2312.483333,1100.750000,14.183180
13323,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),10.694853,NaN,NaN,10.660088,NaN,11.444984,14.790123,18.772727,10.356209,14.632686,9.476190,17.833333,11.033333,13.252033,NaN,NaN,25.986395,11.813063,16.256410,25.833333,15.586667,17.473810,NaN,21.075426,NaN,NaN,16.559829,NaN,21.588050,NaN,11.688889,NaN,NaN,10.767196,NaN,NaN,11.762452,NaN,NaN,2581.333333,1344.616667,15.188612
18340,2008,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),NaN,4.723913,NaN,NaN,8.472222,9.040453,12.572016,16.218182,9.852941,15.737864,11.806122,21.025000,8.951852,8.869919,NaN,NaN,25.017007,19.596847,13.961538,26.729630,10.395000,14.508333,27.432660,NaN,NaN,21.005291,NaN,NaN,24.845912,NaN,17.430556,NaN,30.506494,NaN,NaN,15.212121,NaN,NaN,18.905405,2707.316667,1450.333333,16.367387
3763,2016,00a7ade381ebb6d25e1aacb0bc7129e9,Male,Masters Men 1 (40-49),8.912990,NaN,NaN,7.186404,NaN,10.422330,13.958848,19.400000,11.287582,16.563107,26.071429,16.337500,10.370370,11.008130,48.0,NaN,22.217687,14.506757,17.227564,21.659259,11.713333,14.100000,NaN,NaN,22.088496,NaN,NaN,15.302721,25.490566,NaN,13.372222,NaN,25.939394,NaN,NaN,20.959596,NaN,NaN,15.957207,2703.916667,1383.883333,17.602140
17546,2008,00b39183bebaffe426e376f7efc8ae1b,Male,Masters Men 1 (40-49),NaN,NaN,NaN,NaN,29.197222,10.140777,13.462963,17.600000,11.225490,14.611650,11.180272,17.779167,10.611111,10.662602,NaN,NaN,22.261905,12.840090,13.230769,20.633333,10.486667,11.557143,17.387205,NaN,NaN,15.240741,NaN,NaN,20.226415,NaN,13.408333,NaN,19.309524,NaN,NaN,11.808081,NaN,NaN,9.322072,2255.450000,998.466667,14.964501


In [199]:
# Merge to get non-pb runs with personal best time (with time, timediff and AVG_Pace)
merge = npb.merge(pb,how='left',on='name_mask',suffixes=('', '_pb'))

In [200]:
#merge = merge.groupby('name_mask').head(1)

In [201]:
merge.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
0,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,2312.483333,1100.750000,14.183180,Male,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
1,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),10.694853,NaN,NaN,10.660088,NaN,11.444984,14.790123,18.772727,10.356209,14.632686,9.476190,17.833333,11.033333,13.252033,NaN,NaN,25.986395,11.813063,16.256410,25.833333,15.586667,17.473810,NaN,21.075426,NaN,NaN,16.559829,NaN,21.588050,NaN,11.688889,NaN,NaN,10.767196,NaN,NaN,11.762452,NaN,NaN,2581.333333,1344.616667,15.188612,Male,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
2,2008,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),NaN,4.723913,NaN,NaN,8.472222,9.040453,12.572016,16.218182,9.852941,15.737864,11.806122,21.025000,8.951852,8.869919,NaN,NaN,25.017007,19.596847,13.961538,26.729630,10.395000,14.508333,27.432660,NaN,NaN,21.005291,NaN,NaN,24.845912,NaN,17.430556,NaN,30.506494,NaN,NaN,15.212121,NaN,NaN,18.905405,2707.316667,1450.333333,16.367387,Male,8.261029,NaN,NaN,7.385965,NaN,9.022654,11.744856,15.790909,8.666667,13.215210,8.047619,16.012500,9.270370,11.483740,NaN,NaN,30.527211,10.997748,12.743590,21.881481,13.003333,16.636905,NaN,21.5693

In [202]:
merge.loc[1613]

Year                                                           2015
name_mask                          fbac5ac0b66892696a502c9f3de3ee0e
gender                                                         Male
category                                          Youth Men (20-22)
Chamonix to Delevret                                        7.48897
Chamonix to La Charme                                           NaN
Chamonix to St-Gervais                                          NaN
Delevret to St-Gervais                                      5.99342
La Charme to St-Gervais                                         NaN
St-Gervais to Contamines                                    7.80583
Contamines to La Balme                                      9.45885
La Balme to Bonhomme                                        14.3242
Bonhomme to Chapieux                                        8.26471
Chapieux to Col Seigne                                      11.6942
Col Seigne to Lac Combal                        

In [203]:
merge.groupby('category').count()

,Year,name_mask,gender,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
category,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Masters Men 1 (40-49),690,690,690,484,65,96,484,88,687,688,689,690,689,688,687,674,689,144,77,612,687,686,689,690,690,289,122,279,289,122,279,632,57,633,57,510,122,57,510,122,57,568,690,690,690,690,509,56,113,509,60,685,689,686,689,688,690,687,677,690,106,59,631,685,682,689,690,690,308,92,290,308,92,288,610,77,613,77,521,92,76,521,92,77,598,690,690,690
Masters Men 2 (50-59),290,290,290,233,14,30,233,22,290,289,290,290,289,290,288,288,288,79,39,250,289,290,290,290,290,99,42,148,99,42,149,255,34,256,34,213,42,34,213,42,34,248,290,290,290,290,167,33,67,168,51,286,289,286,288,290,290,290,283,290,32,24,265,290,283,290,290,290,164,39,87,164,39,86,268,22,268,22,228,39,22,229,39,22,251,290,290,290
Masters Men 3 (60-69),31,31,31,26,5,0,25,4,31,31,31,31,31,31,31,31,31,7,4,26,31,31,31,31,31,8,6,17,8,6,17,26,5,26,5,20,6,5,20,6,5,25,31,31,31,31,12,0,15,12,4,31,31,30,31,31,31,31,31,31,0,0,31,31,31,31,31,31,23,6,2,23,6,2,30,1,30,1,24,6,1,24,6,1,25,31,31,31
Masters Men 4 (70-79),1,1,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,1,1,1,1,1,0,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,1,1,1,1
Masters Women 1 (40-49),46,46,46,35,3,6,35,4,46,46,46,46,46,46,45,46,46,16,9,36,46,46,46,46,46,16,3,27,16,3,27,43,3,43,3,40,3,3,40,3,3,43,46,46,46,46,38,1,7,38,1,46,46,46,46,46,46,46,46,46,10,3,43,46,46,46,46,46,13,6,27,13,6,27,36,10,36,10,30,6,10,30,6,10,40,46,46,46
Masters Women 2 (50-59),14,14,14,11,1,2,11,0,14,14,14,14,14,14,14,14,14,4,1,13,14,14,14,14,14,5,2,7,5,2,7,12,2,12,2,10,2,2,10,2,2,12,14,14,14,14,12,0,1,12,1,14,14,14,14,14,14,14,14,14,0,0,14,14,14,14,14,14,7,2,5,7,2,5,13,1,13,1,11,2,1,11,2,1,12,14,14,14
Masters Women 3 (60-69),2,2,2,2,0,0,2,0,2,2,2,2,2,2,2,2,2,1,1,1,2,2,2,2,2,0,0,2,0,0,2,2,0,2,0,2,0,0,2,0,0,2,2,2,2,2,2,0,0,2,0,2,2,2,2,2,2,2,2,2,0,0,2,2,2,2,2,2,2,0,0,2,0,0,2,0,2,0,2,0,0,2,0,0,2,2,2,2
Senior Men (23-39),501,501,501,318,59,89,319,84,497,499,500,500,499,499,

In [204]:
#merge = merge[merge.category != 'Youth Men (20-22)']
#merge = merge[merge.category != 'Masters Women 3 (60-69)']
#merge = merge[merge.category != 'Masters Men 4 (70-79)']

In [205]:
#df = df[~df["AVG_Pace"] > 12 & df.category == 'Masters Men 3 (60-69)']

merge[(merge["AVG_Pace"] <12) & (merge.category == 'Masters Women 2 (50-59)')]



#df[(df["AVG_Pace"] <12) & (df.category == 'Masters Women 2 (50-59)')]


,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
347,2016,36da23ac15e20c3b3e35b879229da995,Female,Masters Women 2 (50-59),7.949755,NaN,NaN,5.844298,NaN,8.763754,9.91358,13.748485,7.372549,10.580906,14.908163,13.258333,7.566667,8.939024,18.055556,NaN,14.758503,8.916667,8.650641,15.911111,6.843333,8.625,NaN,NaN,11.520649,NaN,NaN,8.564626,16.606918,NaN,8.194444,NaN,17.140693,NaN,NaN,14.343434,NaN,NaN,8.497748,1753.933333,433.9,11.018994,Female,7.545343,NaN,NaN,6.116228,NaN,8.108414,9.133745,14.109091,7.745098,10.985437,6.323129,12.966667,7.181481,8.353659,NaN,NaN,16.510204,9.153153,8.86859,16.151852,7.035,8.82619,NaN,NaN,11.446903,NaN,NaN,7.55102,15.613208,NaN,7.3,NaN,15.718615,NaN,NaN,11.636364,NaN,NaN,9.177928,1667.866667,456.133333,10.148222


In [206]:
merge = merge.drop([30])
merge = merge.drop([347])

In [207]:
merge = merge[merge.name_mask != '4a40c92f6b52512ac605ea7bbcec06b6']
merge = merge[merge.name_mask != '4d5f9076cc4be3cdf23a51be2c1e7d8c']
merge = merge[merge.name_mask != '738138c60e7b32635a37f6d0757c1512']
merge = merge[merge.name_mask != '972253ab3ee99e5c6bcee7a8eba23ebf']
merge = merge[merge.name_mask != 'fbac5ac0b66892696a502c9f3de3ee0e']

In [208]:
merge = merge.reset_index(drop=True)

In [209]:
merge.loc[1606]

Year                                                           2013
name_mask                          fc39b99cac070ba6abcc7ca7e959621b
gender                                                         Male
category                                      Masters Men 1 (40-49)
Chamonix to Delevret                                        9.36642
Chamonix to La Charme                                           NaN
Chamonix to St-Gervais                                          NaN
Delevret to St-Gervais                                      8.05482
La Charme to St-Gervais                                         NaN
St-Gervais to Contamines                                    12.8382
Contamines to La Balme                                      12.5761
La Balme to Bonhomme                                        19.7939
Bonhomme to Chapieux                                        11.7974
Chapieux to Col Seigne                                      14.8333
Col Seigne to Lac Combal                        

In [210]:
cats = merge['category']
gen = merge['gender']
comb = [cats, gen]
comb = pd.concat(comb, axis=1)

In [211]:
# Update pb and npb to use merged indices
# pb does not contain name_mask, category or year
pb = merge.loc[:, merge.columns.str.contains('pb')]
npb = merge.loc[:, ~merge.columns.str.contains('pb')]

In [212]:
merge.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
0,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,2312.483333,1100.750000,14.183180,Male,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
1,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),10.694853,NaN,NaN,10.660088,NaN,11.444984,14.790123,18.772727,10.356209,14.632686,9.476190,17.833333,11.033333,13.252033,NaN,NaN,25.986395,11.813063,16.256410,25.833333,15.586667,17.473810,NaN,21.075426,NaN,NaN,16.559829,NaN,21.588050,NaN,11.688889,NaN,NaN,10.767196,NaN,NaN,11.762452,NaN,NaN,2581.333333,1344.616667,15.188612,Male,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
2,2008,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),NaN,4.723913,NaN,NaN,8.472222,9.040453,12.572016,16.218182,9.852941,15.737864,11.806122,21.025000,8.951852,8.869919,NaN,NaN,25.017007,19.596847,13.961538,26.729630,10.395000,14.508333,27.432660,NaN,NaN,21.005291,NaN,NaN,24.845912,NaN,17.430556,NaN,30.506494,NaN,NaN,15.212121,NaN,NaN,18.905405,2707.316667,1450.333333,16.367387,Male,8.261029,NaN,NaN,7.385965,NaN,9.022654,11.744856,15.790909,8.666667,13.215210,8.047619,16.012500,9.270370,11.483740,NaN,NaN,30.527211,10.997748,12.743590,21.881481,13.003333,16.636905,NaN,21.5693

In [213]:
npb.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
0,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,2312.483333,1100.750000,14.183180
1,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),10.694853,NaN,NaN,10.660088,NaN,11.444984,14.790123,18.772727,10.356209,14.632686,9.476190,17.833333,11.033333,13.252033,NaN,NaN,25.986395,11.813063,16.256410,25.833333,15.586667,17.473810,NaN,21.075426,NaN,NaN,16.559829,NaN,21.588050,NaN,11.688889,NaN,NaN,10.767196,NaN,NaN,11.762452,NaN,NaN,2581.333333,1344.616667,15.188612
2,2008,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),NaN,4.723913,NaN,NaN,8.472222,9.040453,12.572016,16.218182,9.852941,15.737864,11.806122,21.025000,8.951852,8.869919,NaN,NaN,25.017007,19.596847,13.961538,26.729630,10.395000,14.508333,27.432660,NaN,NaN,21.005291,NaN,NaN,24.845912,NaN,17.430556,NaN,30.506494,NaN,NaN,15.212121,NaN,NaN,18.905405,2707.316667,1450.333333,16.367387
3,2016,00a7ade381ebb6d25e1aacb0bc7129e9,Male,Masters Men 1 (40-49),8.912990,NaN,NaN,7.186404,NaN,10.422330,13.958848,19.400000,11.287582,16.563107,26.071429,16.337500,10.370370,11.008130,48.0,NaN,22.217687,14.506757,17.227564,21.659259,11.713333,14.100000,NaN,NaN,22.088496,NaN,NaN,15.302721,25.490566,NaN,13.372222,NaN,25.939394,NaN,NaN,20.959596,NaN,NaN,15.957207,2703.916667,1383.883333,17.602140
4,2008,00b39183bebaffe426e376f7efc8ae1b,Male,Masters Men 1 (40-49),NaN,NaN,NaN,NaN,29.197222,10.140777,13.462963,17.600000,11.225490,14.611650,11.180272,17.779167,10.611111,10.662602,NaN,NaN,22.261905,12.840090,13.230769,20.633333,10.486667,11.557143,17.387205,NaN,NaN,15.240741,NaN,NaN,20.226415,NaN,13.408333,NaN,19.309524,NaN,NaN,11.808081,NaN,NaN,9.322072,2255.450000,998.466667,14.964501


# Define Functions

In [214]:
def find_similar_times(index):
    query = npb.iloc[index]
    same_category = npb[npb.category == query.category]
    similar_times = same_category[(same_category.AVG_Pace > query.AVG_Pace - pace_threshold) & (same_category.AVG_Pace < query.AVG_Pace + pace_threshold)]
    #find all runners either 0.5 minutes/kilometre faster or slower than the query runner
   # if len(similar_times) = 1:
    #    similar_times
    return similar_times
    #else:
     #   return None

# Show sample output
find_similar_times(30)

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
30,2011,050587e57898af56e2b0f1fae0bb00bc,Male,Masters Men 2 (50-59),7.736520,NaN,NaN,6.592105,NaN,7.711974,9.174897,14.442424,8.179739,11.153722,6.962585,13.345833,8.144444,9.154472,NaN,NaN,16.350340,10.306306,10.221154,17.344444,11.898333,10.764286,NaN,16.347932,NaN,NaN,17.104701,NaN,18.358491,NaN,11.980556,NaN,NaN,10.116402,NaN,NaN,9.91954,NaN,NaN,1940.650000,703.933333,11.448313
136,2014,16cb844b642e942d0c64b6771efe16da,Male,Masters Men 2 (50-59),6.890931,NaN,NaN,5.894737,NaN,7.679612,9.310700,13.978788,8.294118,10.639159,7.585034,13.912500,9.551852,12.016260,NaN,NaN,19.020408,10.211712,10.458333,16.566667,10.101667,11.113095,NaN,NaN,13.498525,NaN,NaN,10.346939,14.808176,NaN,8.700000,NaN,15.506494,NaN,NaN,14.590909,NaN,NaN,9.101351,1829.983333,618.250000,11.240749
628,2016,61dc368d00f66e14ac588af1f3f3cc5a,Male,Masters Men 2 (50-59),6.937500,NaN,NaN,5.109649,NaN,7.771845,8.870370,13.300000,6.794118,10.542071,14.884354,14.058333,8.077778,9.914634,22.018519,NaN,17.159864,9.849099,10.519231,16.477778,7.538333,9.745238,NaN,NaN,13.303835,NaN,NaN,9.428571,17.805031,NaN,9.058333,NaN,18.740260,NaN,NaN,15.853535,NaN,NaN,8.979730,1833.400000,513.366667,11.709520
690,2017,6e2e95d07da14d588e518cd7a53c3886,Male,Masters Men 2 (50-59),7.656863,NaN,NaN,6.186404,NaN,7.977346,11.185185,15.872727,8.957516,13.464401,8.799320,15.916667,7.218519,7.813008,NaN,NaN,21.204082,9.689189,9.801282,20.907407,7.468333,10.002381,NaN,NaN,14.908555,NaN,NaN,9.840136,NaN,17.645833,NaN,13.600575,NaN,NaN,19.391892,NaN,NaN,18.495238,8.387387,1945.800000,803.900000,12.182927
791,2013,7cee63fbb1e48d49ead414037f92529d,Male,Masters Men 2 (50-59),6.938725,NaN,NaN,5.510965,NaN,8.809061,9.746914,14.654545,8.000000,12.067961,7.261905,15.616667,8.603704,9.898374,NaN,NaN,24.047619,10.882883,10.961538,16.196296,8.446667,10.964286,13.723906,NaN,NaN,12.042328,NaN,NaN,17.198113,NaN,9.683333,NaN,16.982684,NaN,NaN,11.762626,NaN,NaN,9.403153,1895.350000,660.400000,11.641844
1091,2013,a91421c70e86e99c43cef293e6d6d837,Male,Masters Men 2 (50-59),7.085784,NaN,NaN,5.333333,NaN,8.907767,10.300412,16.451515,7.346405,12.190939,6.489796,15.070833,7.777778,9.463415,NaN,NaN,19.846939,9.786036,10.541667,20.307407,8.015000,10.180952,13.092593,NaN,NaN,10.917989,NaN,NaN,19.062893,NaN,7.863889,NaN,19.549784,NaN,NaN,11.959596,NaN,NaN,9.923423,1880.083333,645.133333,11.561089
1093,2009,a91421c70e86e99c43cef293e6d6d837,Male,Masters Men 2 (50-59),NaN,NaN,6.920930,NaN,NaN,11.258900,15.002058,20.269697,9.222222,13.608414,6.727891,17.345833,6.503704,7.215447,NaN,NaN,18.017007,9.346847,9.028846,21.218519,7.886667,10.145238,12.186869,NaN,NaN,9.521164,NaN,NaN,17.641509,NaN,10.094444,NaN,18.147186,NaN,NaN,11.530303,NaN,NaN,8.340090,1941.150000,647.850000,12.051295
1154,2014,b46d3adf6faea7fa1cafbb9cd743c269,Male,Masters Men 2 (50-59),7.035539,NaN,NaN,6.153509,NaN,8.310680,10.716049,15.500000,7.663399,12.813916,7.734694,17.508333,8.225926,9.544715,NaN,NaN,22.176871,10.119369,10.538462,20.011111,8.298333,10.027381,NaN,NaN,15.156342,NaN,NaN,8.605442,16.506289,NaN,8.641667,NaN,15.043290,NaN,NaN,11.09

In [215]:
from scipy.spatial.distance import cosine

In [216]:
def cosine_similarity(a, b):
    # Combine two lists (row and query run) into matrix
    matrix = pd.DataFrame({"A": a, "B": b})
    # Now drop any pairs containing NA so we only compare when we have two times
    matrix = matrix.dropna(axis = 0, how='any')
    # Parse back to two lists
    a = matrix[['A']]
    b = matrix[['B']]
    # Return cosine similarity (1 - cosine difference)
    return 1 - (cosine(a, b))

def sims(index):
    similar_times = find_similar_times(index)
    #similar_times = similar_times.dropna(how='all')
    #if sim_count > 1:
    similar_times = similar_times.filter(like='to')
    #if similar_times.count() > 1:
    query_segs = list(similar_times.loc[index])
    similar_times = similar_times.drop([index], axis = 0)
    result = similar_times.apply(lambda row: cosine_similarity(list(row), query_segs), axis = 1)
    result = result.sort_values(ascending = False) #finding the most similar
    result = result.head(10) #10 is the number of neighbours as an example, the optimal value for n is tested in Prediction_Eval notebook
    
    return result

    #elif sim_count <= 1:
     #   return 0

sims(30)

1219    0.993496
1497    0.992338
136     0.990704
1091    0.988013
1259    0.987984
1374    0.986394
690     0.986086
1154    0.984448
791     0.984309
1341    0.983435
dtype: float64

In [217]:
def weight(index):
    weighter = sims(index)
    #weight_count = len(weighter)
    #weight0 = len(str(weight_count))
   # if weight_count > 1:
    weighter = weighter / weighter.sum()
    return weighter
    
    #elif weight0 <= 1:
     #   return 0
    

weight(30)

1219    0.100585
1497    0.100467
136     0.100302
1091    0.100030
1259    0.100027
1374    0.099866
690     0.099834
1154    0.099669
791     0.099655
1341    0.099566
dtype: float64

In [218]:
#def rank_weight(index):
 #   r_weight = sims(index)
  #  top1 = r_weight.iloc[[0]]*0.25
   # top2 = r_weight.iloc[[1]]*0.25
#    top3 = r_weight.iloc[[2]]*0.125
 #   top4 = r_weight.iloc[[3]]*0.125
 #   top5 = r_weight.iloc[[4]]*0.0625
 #   top6 = r_weight.iloc[[5]]*0.0625
 #   top7 = r_weight.iloc[[6]]*0.03125
  #  top8 = r_weight.iloc[[7]]*0.03125
 #   top9 = r_weight.iloc[[8]]*0.015625
 #   top10 = r_weight.iloc[[9]]*0.0156255
 #   bal = top1.values[0] + top2.values[0] + top3.values[0] + top4.values[0] + top5.values[0] + top6.values[0] + top7.values[0] + top8.values[0] + top9.values[0] + top10.values[0]
 #   return bal

#rank_weight(30)

In [219]:
def find_nearest_neighbours(index):
    #w = weight(index)
    cosine_distance = weight(index)  
    nearest_neighbours = merge.iloc[cosine_distance.index]
    nearest_neighbours['weights'] = weight(index)
    nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
   # nearest_neighbours['rank_weights'] = rank_weight(index)
   # nearest_neighbours['ranked_pace'] = nearest_neighbours['rank_weights'] * nearest_neighbours['AVG_Pace_pb']
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
    return nearest_neighbours
    
# Show sample output    
find_nearest_neighbours(60)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb,weights,weighted_pace
1476,2017,e57a4ab246920b51b977150afa9b0946,Male,Masters Men 1 (40-49),7.357843,NaN,NaN,6.298246,NaN,7.911003,10.598765,16.903030,8.421569,14.548544,7.659864,17.733333,8.292593,8.739837,NaN,NaN,22.850340,12.752252,11.698718,28.007407,8.983333,11.392857,NaN,NaN,17.936578,NaN,NaN,13.445578,NaN,26.559028,NaN,18.250000,NaN,NaN,82.162162,NaN,NaN,20.369048,11.376126,2457.850000,1315.950000,16.677002,Male,7.892157,NaN,NaN,7.109649,NaN,8.902913,11.137860,16.560606,9.225490,13.849515,7.894558,17.075000,9.629630,10.390244,NaN,NaN,22.816327,NaN,25.599359,22.203704,9.610000,11.652381,NaN,NaN,14.640118,NaN,NaN,12.251701,25.182390,NaN,13.769444,NaN,24.045455,NaN,NaN,20.828283,NaN,NaN,13.538288,2216.666667,1004.933333,14.600220,0.100229,1.463364
283,2017,2e9e5af178819103f41abef8c1719f6e,Male,Masters Men 1 (40-49),8.613971,NaN,NaN,7.587719,NaN,10.144013,14.164609,19.078788,10.143791,15.977346,8.656463,19.641667,10.114815,11.146341,NaN,NaN,26.292517,12.639640,13.330128,29.174074,10.985000,16.877381,NaN,NaN,21.330383,NaN,NaN,17.690476,NaN,27.048611,NaN,20.106322,NaN,NaN,23.756757,NaN,NaN,22.623810,12.511261,2609.583333,1467.683333,16.234828,Male,9.382353,NaN,NaN,7.620614,NaN,10.411003,13.090535,17.666667,9.666667,15.111650,18.792517,17.312500,9.544444,9.707317,27.537037,NaN,20.442177,12.806306,12.762821,23.214815,9.856667,14.977381,NaN,NaN,22.210914,NaN,NaN,17.119048,26.839623,NaN,14.572222,NaN,23.712121,NaN,NaN,16.691919,NaN,NaN,10.849099,2526.283333,1206.250000,15.675937,0.100111,1.569334
561,2013,5951c9e449468b2cf2d89cc4a52b20a8,Male,Masters Men 1 (40-49),9.254902,NaN,NaN,8.258772,NaN,12.828479,12.300412,20.472727,12.169935,16.500000,11.204082,20.850000,11.866667,13.097561,NaN,NaN,24.544218,14.515766,17.038462,27.955556,12.885000,17.341667,20.156566,NaN,NaN,21.587302,NaN,NaN,31.418239,NaN,11.808333,NaN,25.413420,NaN,NaN,16.510101,NaN,NaN,13.966216,2750.566667,1515.616667,16.831016,Male,9.174020,NaN,NaN,8.592105,NaN,10.156958,13.115226,18.478788,10.588235,14.352751,9.585034,20.345833,11.851852,17

In [220]:
def find_nearest_neighbours_pace(index):
    #w = weight(index)
    cosine_distance = weight(index)  
    nearest_neighbours = merge.iloc[cosine_distance.index]
    nearest_neighbours['weights'] = weight(index)
    nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
    nearest_neighbours = nearest_neighbours.loc[:, 'Chamonix to Delevret_pb':'Flegere to Chamonix_pb'].mul(nearest_neighbours['weighted_pace'], axis=0)
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
    return nearest_neighbours
    
# Show sample output    
find_nearest_neighbours_pace(60)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb
1476,11.549095,NaN,NaN,10.404002,NaN,13.028199,16.298739,24.234189,13.500247,20.266876,11.552609,24.986934,14.091650,15.204705,NaN,NaN,33.388583,NaN,37.461171,32.492093,14.062925,17.051671,NaN,NaN,21.423816,NaN,NaN,17.928693,36.850994,NaN,20.149704,NaN,35.187244,NaN,NaN,30.479352,NaN,NaN,19.811439
283,14.724046,NaN,NaN,11.959289,NaN,16.338342,20.543422,27.724902,15.170229,23.715228,29.491737,27.169096,14.978422,15.234023,43.214810,NaN,32.080604,20.097373,20.029129,36.431799,15.468403,23.504514,NaN,NaN,34.856344,NaN,NaN,26.865504,42.120334,NaN,22.868685,NaN,37.212239,NaN,NaN,26.195197,NaN,NaN,17.025861
561,14.716600,NaN,NaN,13.783116,NaN,16.293391,21.038928,29.642942,16.985229,23.024116,15.375933,32.637983,19.012273,27.864138,NaN,NaN,40.786054,20.478337,25.548313,44.352068,27.837532,27.286007,NaN,29.302275,NaN,NaN,34.972756,NaN,38.282938,NaN,25.974032,NaN,NaN,21.006864,NaN,NaN,25.675788,NaN,NaN
748,13.873607,NaN,NaN,12.532740,NaN,17.963181,25.670550,32.259829,19.601280,25.994492,15.675881,33.661091,17.201846,23.092547,NaN,NaN,50.685164,22.243851,23.197465,43.206458,18.339397,26.154596,NaN,NaN,33.003775,NaN,NaN,26.106179,44.087156,NaN,21.771934,NaN,38.761373,NaN,NaN,29.413760,NaN,NaN,20.774606
510,16.332490,NaN,NaN,17.308899,NaN,18.570284,20.018014,35.241107,22.388587,26.100742,17.941041,34.551366,21.656443,28.546125,NaN,NaN,40.399896,24.771928,25.756415,41.321701,21.182997,25.220496,NaN,NaN,33.384895,NaN,NaN,28.618480,46.515833,NaN,27.394198,NaN,34.777076,NaN,NaN,33.785127,NaN,NaN,23.257421
1206,14.040268,NaN,NaN,12.582243,NaN,15.340354,18.053734,28.196718,15.681017,22.715665,14.361297,26.634625,14.500864,16.985227,NaN,NaN,42.816417,18.487682,19.572094,35.459624,17.743815,22.824278,NaN,NaN,29.214663,NaN,NaN,23.131358,36.384521,NaN,23.150133,NaN,32.353133,NaN,NaN,29.015475,NaN,NaN,19.768332
1339,15.887425,NaN,NaN,15.053384,NaN,18.798501,24.716431,32.636373,21.148004,27.353764,17.396849,37.128649,19.901453,21.957667,NaN,NaN,49.742351,23.905650,27.359183,43.261091,21.687056,27.050577,NaN,NaN,33.467513,NaN,NaN,30.500895,45.617928,NaN,22.246324,NaN,36.468049,NaN,NaN,28.620903,NaN,NaN,18.485791
482,12.464169,NaN,NaN,10.564101,NaN,14.386592,22.532293,26.755197,16.068290,22.201941,15.427318,34.652678,15.228341,20.752668,NaN,NaN,57.436761,23.291936,21.058014,40.071806,18.709941,25.864031,NaN,NaN,35.996631,NaN,NaN,24.833301,45.962953,NaN,18.839139,NaN,35.126169,NaN,NaN,27.629299,NaN,NaN,15.071881
285,14.695743,NaN,NaN,11.936301,NaN,16.306936,20.503933,27.671608,15.141069,23.669642,29.435047,27.116871,14.949630,15.204740,43.131741,NaN,32.018938,20.058741,19.990628,36.361769,15.438669,23.459333,NaN,NaN,34.789343,NaN,NaN,26.813863,42.039369,NaN,22.824726,NaN,37.140709,NaN,NaN,26.144844,NaN,NaN,16.993133
1591,14.159388,NaN,NaN,11.610823,NaN,19.092787,18.511014,29.657769,17.961592,24.732888,17.295084,31.473788,18.677032,18.498462,NaN,NaN,38.278616,21.770353,24.841602,41.238887,19.149188,23.308296,36.392367,NaN,NaN,29.008099,NaN,NaN,44.584684,NaN,26.917339,NaN,50.768540

In [231]:
def weighted_row(neighbours):
    #row_pace = find_nearest_neighbours_pace(index)
    row_pace = neighbours.mean()
    row_pace = pd.DataFrame(row_pace)
    row_pace = row_pace.T
    #pred_pace = row_pace
    return row_pace

neighbours = find_nearest_neighbours_pace(60)
weighted_row(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb
0,14.244283,NaN,NaN,12.77349,NaN,16.611856,20.788706,29.402063,17.364554,23.977535,18.39528,31.001308,17.019795,20.33403,43.173276,NaN,41.763338,21.678428,24.481401,39.41973,18.961992,24.17238,36.392367,29.302275,32.017122,29.008099,34.972756,25.599784,42.244671,NaN,23.213621,NaN,37.532726,21.006864,NaN,28.491849,25.675788,NaN,19.30245


In [256]:
def weighted_rows(index):
    row_pace = find_nearest_neighbours_pace(index)
    row_pace = row_pace.mean()
    row_pace = pd.DataFrame(row_pace)
    row_pace = row_pace.T
    #pred_pace = row_pace
    return row_pace

#neighbours = find_nearest_neighbours_pace(60)
#weighted_row(neighbours)
weighted_rows(0)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb
0,10.303154,NaN,NaN,8.329211,43.18377,11.823804,14.833333,21.126334,11.49742,17.637171,15.383465,23.740521,11.190492,14.451683,53.982091,28.813351,29.2393,14.291341,16.465956,27.949701,11.833574,14.694346,25.499187,NaN,20.785376,21.039664,NaN,14.229927,28.628679,23.563158,15.540074,18.051462,26.990158,NaN,22.41812,21.343407,NaN,22.22018,12.723819


In [267]:
row_weights = pb.filter(like = 'to')
row_weights.apply(weighted_rows)

IndexError: ('positional indexers are out-of-bounds', 'occurred at index Chamonix to Delevret_pb')

In [222]:
def get_mean_pb(neighbours):
    mean = neighbours.AVG_Pace_pb.mean() 
    return mean
    
# Show sample output    
neighbours = find_nearest_neighbours(60)
get_mean_pb(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


15.86120579508146

In [223]:
def get_weight_mean(neighbours):
    w_mean = neighbours.weighted_pace.sum()
    return w_mean

neighbours = find_nearest_neighbours(30)
get_weight_mean(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


11.181970610319633

In [224]:
#def get_rank_mean(neighbours):
 #   ranked_weight = neighbours.ranked_pace.mean()
  #  return ranked_weight

#neighbours = find_nearest_neighbours(30)
#get_rank_mean(neighbours)

In [225]:
def get_best_npb(neighbours):
    return neighbours.AVG_Pace.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(60)
get_best_npb(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


16.223586238176683

In [226]:
def get_best_pacer(neighbours):
    b_pace = neighbours.sort_values(by='AVG_Pace_pb', ascending=True)
    return b_pace.head(1)

neighbours = find_nearest_neighbours(60)
get_best_pacer(neighbours)
    

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb,weights,weighted_pace
1476,2017,e57a4ab246920b51b977150afa9b0946,Male,Masters Men 1 (40-49),7.357843,NaN,NaN,6.298246,NaN,7.911003,10.598765,16.90303,8.421569,14.548544,7.659864,17.733333,8.292593,8.739837,NaN,NaN,22.85034,12.752252,11.698718,28.007407,8.983333,11.392857,NaN,NaN,17.936578,NaN,NaN,13.445578,NaN,26.559028,NaN,18.25,NaN,NaN,82.162162,NaN,NaN,20.369048,11.376126,2457.85,1315.95,16.677002,Male,7.892157,NaN,NaN,7.109649,NaN,8.902913,11.13786,16.560606,9.22549,13.849515,7.894558,17.075,9.62963,10.390244,NaN,NaN,22.816327,NaN,25.599359,22.203704,9.61,11.652381,NaN,NaN,14.640118,NaN,NaN,12.251701,25.18239,NaN,13.769444,NaN,24.045455,NaN,NaN,20.828283,NaN,NaN,13.538288,2216.666667,1004.933333,14.60022,0.100229,1.463364


In [227]:
def get_best_pb(neighbours):
    return neighbours.AVG_Pace_pb.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(60)
get_best_pb(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


14.60022042361784

In [228]:
def get_prediction(index, neighbours):
    weight = merge.iloc[index].AVG_Pace/get_best_npb(neighbours)
    best_pb = get_best_pb(neighbours)
    return best_pb * weight

# Show output
neighbours = find_nearest_neighbours(0)
get_prediction(0, neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


11.680842484832413

# Calculate Predictions

In [229]:
def update_rows(row):
    index = int(row['best_pb_prediction'])
    row['query_AVG_Pace'] = merge.iloc[index].AVG_Pace
    row['query_pb'] = merge.iloc[index].AVG_Pace_pb
    neighbours = find_nearest_neighbours(index)
    row['mean_pb_average_pace'] = get_mean_pb(neighbours)
   # row['weighted_mean_pb_ap'] = weighted_pace(neighbours)
  #  row['best_average_pace'] = get_best_npb(neighbours)
    row['best_pb_prediction'] = get_prediction(index, neighbours)
    row['weighted_mean'] = get_weight_mean(neighbours)
   # row['ranked_mean'] = get_rank_mean(neighbours)
    return row

In [230]:
num_of_runs = merge.time.count()
filler_data = np.arange(0.0, num_of_runs)
df = pd.DataFrame({'best_pb_prediction':filler_data, 'mean_pb_average_pace':filler_data,  'weighted_mean':filler_data,  'query_AVG_Pace':filler_data, 'query_pb':filler_data})

df = df.apply(lambda row: update_rows(row), axis=1)
df = df[['query_AVG_Pace', 'query_pb', 'best_pb_prediction', 'mean_pb_average_pace', 'weighted_mean']]
df

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


KeyboardInterrupt: 

Comb dataframe contains the information of the gender and category of each runner. This will be used in a later notebook to test how accurate the predictions are in comparison to male, female, senior men, etc...

In [ ]:
comb.join(df)

In [ ]:
df = comb.join(df)

In [ ]:
df.to_csv('../../data/pace_predictions.csv')

# Pace Predictions

In [ ]:
def update_pace(test):
   # index = int(row['best_pb_prediction'])
   # row['query_AVG_Pace'] = merge.iloc[index].AVG_Pace
   # row['query_pb'] = merge.iloc[index].AVG_Pace_pb
   # neighbours = find_nearest_neighbours(index)
   # row['mean_pb_average_pace'] = get_mean_pb(neighbours)
  #  row['best_average_pace'] = get_best_npb(neighbours)
  #  row['best_pb_prediction'] = get_prediction(index, neighbours)
    test = get_best_pacer(neighbours)
    return test

In [ ]:
#num_of_runs = merge.time.count()
#filler_data = np.arange(0.0, num_of_runs)
#df_pace = pd.DataFrame({'best_pacer':filler_data})

#df_pace = df_pace.apply(lambda test: update_pace(test), axis=1)
#df_pace = df_pace[['best_pacer']]
#df_pace

# ******NEEDS FIXING*******

In [ ]:
def Cham_to_Del(neighbours):
    return neighbours['Chamonix to Delevret_pb'].mean()

def Cham_to_LaCh(neighbours):
    return neighbours['Chamonix to La Charme_pb'].mean()

def Cham_to_StG(neighbours):
    return neighbours['Chamonix to St-Gervais_pb'].mean()

def Del_to_StG(neighbours):
    return neighbours['Delevret to St-Gervais_pb'].mean()

def LaCh_to_StG(neighbours):
    return neighbours['La Charme to St-Gervais_pb'].mean()

def StG_to_Cont(neighbours):
    return neighbours['St-Gervais to Contamines_pb'].mean()

def Cont_to_LaB(neighbours):
    return neighbours['Contamines to La Balme_pb'].mean()

def LaB_to_Bon(neighbours):
    return neighbours['La Balme to Bonhomme_pb'].mean()

def Bon_to_Chap(neighbours):
    return neighbours['Bonhomme to Chapieux_pb'].mean()

def Chap_to_ColS(neighbours):
    return neighbours['Chapieux to Col Seigne_pb'].mean()

def ColS_to_Lac(neighbours):
    return neighbours['Col Seigne to Lac Combal_pb'].mean()

def Lac_to_MtF(neighbours):
    return neighbours['Lac Combal to Mt-Favre_pb'].mean()

def MtF_to_Chec(neighbours):
    return neighbours['Mt-Favre to Checruit_pb'].mean()

def Chec_to_Courm(neighbours):
    return neighbours['Checruit to Courmayeur_pb'].mean()

def Courm_to_Courm2(neighbours):
    return neighbours['Courmayeur to Courmayeur2_pb'].mean()

def Courm2_to_Bertone(neighbours):
    return neighbours['Courmayeur2 to Bertone_pb'].mean()

def Courm_to_Bertone(neighbours):
    return neighbours['Courmayeur to Bertone_pb'].mean()

def Bertone_to_Bon(neighbours):
    return neighbours['Bertone to Bonatti_pb'].mean()

def Bon_to_Arn(neighbours):
    return neighbours['Bonatti to Arnouvaz_pb'].mean()

def Arn_to_Col(neighbours):
    return neighbours['Arnouvaz to Col Ferret_pb'].mean()

def Col_to_LaF(neighbours):
    return neighbours['Col Ferret to La Fouly_pb'].mean()

def LaF_to_Champ(neighbours):
    return neighbours['La Fouly to Champex La_pb'].mean()

def Champ_to_Bov(neighbours):
    return neighbours['Champex La to Bovine_pb'].mean()

def Champ_to_Mort(neighbours):
    return neighbours['Champex La to Mortigny_pb'].mean()

def Champ_to_Giete(neighbours):
    return neighbours['Champex La to Giete_pb'].mean()

def Bov_to_Trient(neighbours):
    return neighbours['Bovine to Trient_pb'].mean()

def Mort_to_Trient(neighbours):
    return neighbours['Mortigny to Trient_pb'].mean()

def Giete_to_Trient(neighbours):
    return neighbours['Giete to Trient_pb'].mean()

def Trient_to_LaCa(neighbours):
    return neighbours['Trient to La Catogne_pb'].mean()

def Trient_to_LesSt(neighbours):
    return neighbours['Trient to Les Tseppe_pb'].mean()

def LaCa_to_Vall(neighbours):
    return neighbours['La Catogne to Vallorcine_pb'].mean()

def LesSt_to_Vall(neighbours):
    return neighbours['Les Tseppe to Vallorcine_pb'].mean()

def Vall_to_tAV(neighbours):
    return neighbours['Vallorcine to Tete aux Vents_pb'].mean()

def Vall_to_Arg(neighbours):
    return neighbours['Vallorcine to Argentiere_pb'].mean()

def Vall_to_Col(neighbours):
    return neighbours['Vallorcine to Col Montet_pb'].mean()

def tAV_to_Fleg(neighbours):
    return neighbours['Tete aux Vents to Flegere_pb'].mean()

def Arg_to_Cham(neighbours):
    return neighbours['Argentiere to Chamonix_pb'].mean()

def Col_to_Fleg(neighbours):
    return neighbours['Col Montet to Flegere_pb'].mean()

def Fleg_to_Cham(neighbours):
    return neighbours['Flegere to Chamonix_pb'].mean()

# Show sample output    
neighbours = find_nearest_neighbours(0)
#neighbours = neighbours.filter(like='pb')
#get_best_pace(neighbours)
Cham_to_StG(neighbours)
#Cham_to_LaCh(neighbours)
#Cham_to_StG(neighbours)   


In [ ]:
def get_prediction(index, neighbours):
    weight = merge.iloc[index].AVG_Pace/get_best_npb(neighbours)
    best_pb = get_best_pb(neighbours)
    return best_pb * weight

# Show output
neighbours = find_nearest_neighbours(0)
get_prediction(0, neighbours)

# Mean Pace Recommendations

In [ ]:
def update_pace(line):
    index = int(line['Chamonix to Delevret_pb'])
    neighbours = find_nearest_neighbours(index)
    line['Chamonix to St-Gervais_pb'] = Cham_to_StG(neighbours)
    line['Chamonix to La Charme_pb'] = Cham_to_LaCh(neighbours)
    line['Delevret to St-Gervais_pb'] = Del_to_StG(neighbours)
    line['La Charme to St-Gervais_pb'] = LaCh_to_StG(neighbours)
    line['St-Gervais to Contamines_pb'] = StG_to_Cont(neighbours)
    line['Contamines to La Balme_pb'] = Cont_to_LaB(neighbours)
    line['La Balme to Bonhomme_pb'] = LaB_to_Bon(neighbours)
    line['Bonhomme to Chapieux_pb'] = Bon_to_Chap(neighbours)
    line['Chapieux to Col Seigne_pb'] = Chap_to_ColS(neighbours)
    line['Col Seigne to Lac Combal_pb'] = ColS_to_Lac(neighbours)
    line['Lac Combal to Mt-Favre_pb'] = Lac_to_MtF(neighbours)
    line['Mt-Favre to Checruit_pb '] = MtF_to_Chec(neighbours)
    line['Checruit to Courmayeur_pb'] = Chec_to_Courm(neighbours)
    line['Courmayeur to Courmayeur2_pb'] = Courm_to_Courm2(neighbours)
    line['Courmayeur2 to Bertone_pb'] = Courm2_to_Bertone(neighbours)
    line['Courmayeur to Bertone_pb'] = Courm_to_Bertone(neighbours)
    line['Bertone to Bonatti_pb'] = Bertone_to_Bon(neighbours)
    line['Bonatti to Arnouvaz_pb'] = Bon_to_Arn(neighbours)
    line['Arnouvaz to Col Ferret_pb'] = Arn_to_Col(neighbours)
    line['Col Ferret to La Fouly_pb'] = Col_to_LaF(neighbours)
    line['La Fouly to Champex La_pb'] = LaF_to_Champ(neighbours)
    line['Champex La to Bovine_pb'] = Champ_to_Bov(neighbours)
    line['Champex La to Mortigny_pb'] = Champ_to_Mort(neighbours)
    line['Champex La to Giete_pb'] = Champ_to_Giete(neighbours)
    line['Bovine to Trient_pb'] = Bov_to_Trient(neighbours)
    line['Mortigny to Trient_pb'] = Mort_to_Trient(neighbours)
    line['Giete to Trient_pb'] = Giete_to_Trient(neighbours)
    line['Trient to La Catogne_pb'] = Trient_to_LaCa(neighbours)
    line['Trient to Les Tseppe_pb'] = Trient_to_LesSt(neighbours)
    line['La Catogne to Vallorcine_pb'] = LaCa_to_Vall(neighbours)
    line['Les Tseppe to Vallorcine_pb'] = LesSt_to_Vall(neighbours)
    line['Vallorcine to Tete aux Vents_pb'] = Vall_to_tAV(neighbours)
    line['Vallorcine to Argentiere_pb'] = Vall_to_Arg(neighbours)
    line['Vallorcine to Col Montet_pb'] = Vall_to_Col(neighbours)
    line['Tete aux Vents to Flegere_pb'] = tAV_to_Fleg(neighbours)
    line['Argentiere to Chamonix_pb'] = Arg_to_Cham(neighbours)
    line['Col Montet to Flegere_pb'] = Col_to_Fleg(neighbours)
    line['Flegere to Chamonix_pb'] = Fleg_to_Cham(neighbours)
    
    line['Chamonix to Delevret_pb'] = Cham_to_Del(neighbours)
    
    return line

In [ ]:
num_of_runs1 = merge.time.count()
filler_data = np.arange(0.0, num_of_runs1)
df1 = pd.DataFrame(
    { 
     'Chamonix to Delevret_pb':filler_data,            
'Chamonix to La Charme_pb':filler_data,            
     'Chamonix to St-Gervais_pb':filler_data,      
'Delevret to St-Gervais_pb':filler_data,           
'La Charme to St-Gervais_pb':filler_data,          
'St-Gervais to Contamines_pb':filler_data,         
'Contamines to La Balme_pb':filler_data,           
'La Balme to Bonhomme_pb':filler_data,             
     'Bonhomme to Chapieux_pb':filler_data,              
'Chapieux to Col Seigne_pb':filler_data,           
'Col Seigne to Lac Combal_pb':filler_data,         
'Lac Combal to Mt-Favre_pb':filler_data,           
'Mt-Favre to Checruit_pb':filler_data,             
'Checruit to Courmayeur_pb':filler_data,           
'Courmayeur to Courmayeur2_pb':filler_data,        
'Courmayeur2 to Bertone_pb':filler_data,           
'Courmayeur to Bertone_pb':filler_data,           
'Bertone to Bonatti_pb':filler_data,               
'Bonatti to Arnouvaz_pb':filler_data,              
'Arnouvaz to Col Ferret_pb':filler_data,           
'Col Ferret to La Fouly_pb':filler_data,           
'La Fouly to Champex La_pb':filler_data,           
'Champex La to Bovine_pb':filler_data,             
'Champex La to Mortigny_pb':filler_data,           
'Champex La to Giete_pb':filler_data,             
'Bovine to Trient_pb':filler_data,                 
'Mortigny to Trient_pb':filler_data,               
'Giete to Trient_pb':filler_data,                  
'Trient to La Catogne_pb':filler_data,             
'Trient to Les Tseppe_pb':filler_data,             
'La Catogne to Vallorcine_pb':filler_data,         
'Les Tseppe to Vallorcine_pb':filler_data,         
'Vallorcine to Tete aux Vents_pb':filler_data,     
'Vallorcine to Argentiere_pb':filler_data,         
'Vallorcine to Col Montet_pb':filler_data,         
'Tete aux Vents to Flegere_pb':filler_data,        
'Argentiere to Chamonix_pb':filler_data,           
'Col Montet to Flegere_pb':filler_data,            
'Flegere to Chamonix_pb':filler_data             
    }
)

df1 = df1.apply(lambda line: update_pace(line), axis=1)
df1 = df1[['Chamonix to Delevret_pb',            
'Chamonix to La Charme_pb',            
     'Chamonix to St-Gervais_pb',      
'Delevret to St-Gervais_pb',           
'La Charme to St-Gervais_pb',          
'St-Gervais to Contamines_pb',         
'Contamines to La Balme_pb',           
'La Balme to Bonhomme_pb',             
     'Bonhomme to Chapieux_pb',              
'Chapieux to Col Seigne_pb',           
'Col Seigne to Lac Combal_pb',         
'Lac Combal to Mt-Favre_pb',           
'Mt-Favre to Checruit_pb',             
'Checruit to Courmayeur_pb',           
'Courmayeur to Courmayeur2_pb',        
'Courmayeur2 to Bertone_pb',           
'Courmayeur to Bertone_pb',           
'Bertone to Bonatti_pb',               
'Bonatti to Arnouvaz_pb',              
'Arnouvaz to Col Ferret_pb',           
'Col Ferret to La Fouly_pb',           
'La Fouly to Champex La_pb',           
'Champex La to Bovine_pb',             
'Champex La to Mortigny_pb',           
'Champex La to Giete_pb',             
'Bovine to Trient_pb',                 
'Mortigny to Trient_pb',               
'Giete to Trient_pb',                  
'Trient to La Catogne_pb',             
'Trient to Les Tseppe_pb',             
'La Catogne to Vallorcine_pb',         
'Les Tseppe to Vallorcine_pb',         
'Vallorcine to Tete aux Vents_pb',     
'Vallorcine to Argentiere_pb',         
'Vallorcine to Col Montet_pb',         
'Tete aux Vents to Flegere_pb',        
'Argentiere to Chamonix_pb',           
'Col Montet to Flegere_pb',            
'Flegere to Chamonix_pb']]
df1.head()

# Best Pace Recommendations

In [ ]:
def Cham_2_Del(neighbours):
    return neighbours['Chamonix to Delevret_pb'].min()

def Cham_2_LaCh(neighbours):
    return neighbours['Chamonix to La Charme_pb'].min()

def Cham_2_StG(neighbours):
    return neighbours['Chamonix to St-Gervais_pb'].min()

def Del_2_StG(neighbours):
    return neighbours['Delevret to St-Gervais_pb'].min()

def LaCh_2_StG(neighbours):
    return neighbours['La Charme to St-Gervais_pb'].min()

def StG_2_Cont(neighbours):
    return neighbours['St-Gervais to Contamines_pb'].min()

def Cont_2_LaB(neighbours):
    return neighbours['Contamines to La Balme_pb'].min()

def LaB_2_Bon(neighbours):
    return neighbours['La Balme to Bonhomme_pb'].min()

def Bon_2_Chap(neighbours):
    return neighbours['Bonhomme to Chapieux_pb'].min()

def Chap_2_ColS(neighbours):
    return neighbours['Chapieux to Col Seigne_pb'].min()

def ColS_2_Lac(neighbours):
    return neighbours['Col Seigne to Lac Combal_pb'].min()

def Lac_2_MtF(neighbours):
    return neighbours['Lac Combal to Mt-Favre_pb'].min()

def MtF_2_Chec(neighbours):
    return neighbours['Mt-Favre to Checruit_pb'].min()

def Chec_2_Courm(neighbours):
    return neighbours['Checruit to Courmayeur_pb'].min()

def Courm_2_Courm2(neighbours):
    return neighbours['Courmayeur to Courmayeur2_pb'].min()

def Courm2_2_Bertone(neighbours):
    return neighbours['Courmayeur2 to Bertone_pb'].min()

def Courm_2_Bertone(neighbours):
    return neighbours['Courmayeur to Bertone_pb'].min()

def Bertone_2_Bon(neighbours):
    return neighbours['Bertone to Bonatti_pb'].min()

def Bon_2_Arn(neighbours):
    return neighbours['Bonatti to Arnouvaz_pb'].min()

def Arn_2_Col(neighbours):
    return neighbours['Arnouvaz to Col Ferret_pb'].min()

def Col_2_LaF(neighbours):
    return neighbours['Col Ferret to La Fouly_pb'].min()

def LaF_2_Champ(neighbours):
    return neighbours['La Fouly to Champex La_pb'].min()

def Champ_2_Bov(neighbours):
    return neighbours['Champex La to Bovine_pb'].min()

def Champ_2_Mort(neighbours):
    return neighbours['Champex La to Mortigny_pb'].min()

def Champ_2_Giete(neighbours):
    return neighbours['Champex La to Giete_pb'].min()

def Bov_2_Trient(neighbours):
    return neighbours['Bovine to Trient_pb'].min()

def Mort_2_Trient(neighbours):
    return neighbours['Mortigny to Trient_pb'].min()

def Giete_2_Trient(neighbours):
    return neighbours['Giete to Trient_pb'].min()

def Trient_2_LaCa(neighbours):
    return neighbours['Trient to La Catogne_pb'].min()

def Trient_2_LesSt(neighbours):
    return neighbours['Trient to Les Tseppe_pb'].min()

def LaCa_2_Vall(neighbours):
    return neighbours['La Catogne to Vallorcine_pb'].min()

def LesSt_2_Vall(neighbours):
    return neighbours['Les Tseppe to Vallorcine_pb'].min()

def Vall_2_tAV(neighbours):
    return neighbours['Vallorcine to Tete aux Vents_pb'].min()

def Vall_2_Arg(neighbours):
    return neighbours['Vallorcine to Argentiere_pb'].min()

def Vall_2_Col(neighbours):
    return neighbours['Vallorcine to Col Montet_pb'].min()

def tAV_2_Fleg(neighbours):
    return neighbours['Tete aux Vents to Flegere_pb'].min()

def Arg_2_Cham(neighbours):
    return neighbours['Argentiere to Chamonix_pb'].min()

def Col_2_Fleg(neighbours):
    return neighbours['Col Montet to Flegere_pb'].min()

def Fleg_2_Cham(neighbours):
    return neighbours['Flegere to Chamonix_pb'].min()

# Show sample output    
neighbours = find_nearest_neighbours(0)
#neighbours = neighbours.filter(like='pb')
#get_best_pace(neighbours)
Cham_2_StG(neighbours)
#Cham_2_LaCh(neighbours)
#Cham_2_StG(neighbours)   


In [ ]:
def update_best(sta):
    index = int(sta['Chamonix to Delevret_pb'])
    neighbours = find_nearest_neighbours(index)
    sta['Chamonix to St-Gervais_pb'] = Cham_2_StG(neighbours)
    sta['Chamonix to La Charme_pb'] = Cham_2_LaCh(neighbours)
    sta['Delevret to St-Gervais_pb'] = Del_2_StG(neighbours)
    sta['La Charme to St-Gervais_pb'] = LaCh_2_StG(neighbours)
    sta['St-Gervais to Contamines_pb'] = StG_2_Cont(neighbours)
    sta['Contamines to La Balme_pb'] = Cont_2_LaB(neighbours)
    sta['La Balme to Bonhomme_pb'] = LaB_2_Bon(neighbours)
    sta['Bonhomme to Chapieux_pb'] = Bon_2_Chap(neighbours)
    sta['Chapieux to Col Seigne_pb'] = Chap_2_ColS(neighbours)
    sta['Col Seigne to Lac Combal_pb'] = ColS_2_Lac(neighbours)
    sta['Lac Combal to Mt-Favre_pb'] = Lac_2_MtF(neighbours)
    sta['Mt-Favre to Checruit_pb '] = MtF_2_Chec(neighbours)
    sta['Checruit to Courmayeur_pb'] = Chec_2_Courm(neighbours)
    sta['Courmayeur to Courmayeur2_pb'] = Courm_2_Courm2(neighbours)
    sta['Courmayeur2 to Bertone_pb'] = Courm2_2_Bertone(neighbours)
    sta['Courmayeur to Bertone_pb'] = Courm_2_Bertone(neighbours)
    sta['Bertone to Bonatti_pb'] = Bertone_2_Bon(neighbours)
    sta['Bonatti to Arnouvaz_pb'] = Bon_2_Arn(neighbours)
    sta['Arnouvaz to Col Ferret_pb'] = Arn_2_Col(neighbours)
    sta['Col Ferret to La Fouly_pb'] = Col_2_LaF(neighbours)
    sta['La Fouly to Champex La_pb'] = LaF_2_Champ(neighbours)
    sta['Champex La to Bovine_pb'] = Champ_2_Bov(neighbours)
    sta['Champex La to Mortigny_pb'] = Champ_2_Mort(neighbours)
    sta['Champex La to Giete_pb'] = Champ_2_Giete(neighbours)
    sta['Bovine to Trient_pb'] = Bov_2_Trient(neighbours)
    sta['Mortigny to Trient_pb'] = Mort_2_Trient(neighbours)
    sta['Giete to Trient_pb'] = Giete_2_Trient(neighbours)
    sta['Trient to La Catogne_pb'] = Trient_2_LaCa(neighbours)
    sta['Trient to Les Tseppe_pb'] = Trient_2_LesSt(neighbours)
    sta['La Catogne to Vallorcine_pb'] = LaCa_2_Vall(neighbours)
    sta['Les Tseppe to Vallorcine_pb'] = LesSt_2_Vall(neighbours)
    sta['Vallorcine to Tete aux Vents_pb'] = Vall_2_tAV(neighbours)
    sta['Vallorcine to Argentiere_pb'] = Vall_2_Arg(neighbours)
    sta['Vallorcine to Col Montet_pb'] = Vall_2_Col(neighbours)
    sta['Tete aux Vents to Flegere_pb'] = tAV_2_Fleg(neighbours)
    sta['Argentiere to Chamonix_pb'] = Arg_2_Cham(neighbours)
    sta['Col Montet to Flegere_pb'] = Col_2_Fleg(neighbours)
    sta['Flegere to Chamonix_pb'] = Fleg_2_Cham(neighbours)
    
    sta['Chamonix to Delevret_pb'] = Cham_2_Del(neighbours)
    
    return sta

In [ ]:
num_of_runs2 = merge.time.count()
filler_data = np.arange(0.0, num_of_runs2)
df2 = pd.DataFrame(
    { 
     'Chamonix to Delevret_pb':filler_data,            
'Chamonix to La Charme_pb':filler_data,            
     'Chamonix to St-Gervais_pb':filler_data,      
'Delevret to St-Gervais_pb':filler_data,           
'La Charme to St-Gervais_pb':filler_data,          
'St-Gervais to Contamines_pb':filler_data,         
'Contamines to La Balme_pb':filler_data,           
'La Balme to Bonhomme_pb':filler_data,             
     'Bonhomme to Chapieux_pb':filler_data,              
'Chapieux to Col Seigne_pb':filler_data,           
'Col Seigne to Lac Combal_pb':filler_data,         
'Lac Combal to Mt-Favre_pb':filler_data,           
'Mt-Favre to Checruit_pb':filler_data,             
'Checruit to Courmayeur_pb':filler_data,           
'Courmayeur to Courmayeur2_pb':filler_data,        
'Courmayeur2 to Bertone_pb':filler_data,           
'Courmayeur to Bertone_pb':filler_data,           
'Bertone to Bonatti_pb':filler_data,               
'Bonatti to Arnouvaz_pb':filler_data,              
'Arnouvaz to Col Ferret_pb':filler_data,           
'Col Ferret to La Fouly_pb':filler_data,           
'La Fouly to Champex La_pb':filler_data,           
'Champex La to Bovine_pb':filler_data,             
'Champex La to Mortigny_pb':filler_data,           
'Champex La to Giete_pb':filler_data,             
'Bovine to Trient_pb':filler_data,                 
'Mortigny to Trient_pb':filler_data,               
'Giete to Trient_pb':filler_data,                  
'Trient to La Catogne_pb':filler_data,             
'Trient to Les Tseppe_pb':filler_data,             
'La Catogne to Vallorcine_pb':filler_data,         
'Les Tseppe to Vallorcine_pb':filler_data,         
'Vallorcine to Tete aux Vents_pb':filler_data,     
'Vallorcine to Argentiere_pb':filler_data,         
'Vallorcine to Col Montet_pb':filler_data,         
'Tete aux Vents to Flegere_pb':filler_data,        
'Argentiere to Chamonix_pb':filler_data,           
'Col Montet to Flegere_pb':filler_data,            
'Flegere to Chamonix_pb':filler_data             
    }
)

df2 = df2.apply(lambda sta: update_best(sta), axis=1)
df2 = df2[['Chamonix to Delevret_pb',            
'Chamonix to La Charme_pb',            
     'Chamonix to St-Gervais_pb',      
'Delevret to St-Gervais_pb',           
'La Charme to St-Gervais_pb',          
'St-Gervais to Contamines_pb',         
'Contamines to La Balme_pb',           
'La Balme to Bonhomme_pb',             
     'Bonhomme to Chapieux_pb',              
'Chapieux to Col Seigne_pb',           
'Col Seigne to Lac Combal_pb',         
'Lac Combal to Mt-Favre_pb',           
'Mt-Favre to Checruit_pb',             
'Checruit to Courmayeur_pb',           
'Courmayeur to Courmayeur2_pb',        
'Courmayeur2 to Bertone_pb',           
'Courmayeur to Bertone_pb',           
'Bertone to Bonatti_pb',               
'Bonatti to Arnouvaz_pb',              
'Arnouvaz to Col Ferret_pb',           
'Col Ferret to La Fouly_pb',           
'La Fouly to Champex La_pb',           
'Champex La to Bovine_pb',             
'Champex La to Mortigny_pb',           
'Champex La to Giete_pb',             
'Bovine to Trient_pb',                 
'Mortigny to Trient_pb',               
'Giete to Trient_pb',                  
'Trient to La Catogne_pb',             
'Trient to Les Tseppe_pb',             
'La Catogne to Vallorcine_pb',         
'Les Tseppe to Vallorcine_pb',         
'Vallorcine to Tete aux Vents_pb',     
'Vallorcine to Argentiere_pb',         
'Vallorcine to Col Montet_pb',         
'Tete aux Vents to Flegere_pb',        
'Argentiere to Chamonix_pb',           
'Col Montet to Flegere_pb',            
'Flegere to Chamonix_pb']]
df2.head()

# Test for value of K

I test for different values of k (nearest neighbours) to try and find the optimal value to get the best prediction. The values used are 10 (original), 1, 5, and 20 and they are tested in Eval_Prediction.ipynb

# K = 5 

## Define Functions

In [ ]:
def find_similar_times(index):
    query = npb.iloc[index]
    same_category = npb[npb.category == query.category]
    similar_times = same_category[(same_category.AVG_Pace > query.AVG_Pace - pace_threshold) & (same_category.AVG_Pace < query.AVG_Pace + pace_threshold)]
    return similar_times

# Show sample output
find_similar_times(0)

In [ ]:
def calculate_euclidean_distance(index):
    similar_times = find_similar_times(index)
    similar_times = similar_times.drop(['category', 'name_mask', 'Year'], axis=1)
    similar_times = similar_times.fillna(0)
    euclidean_distance = pd.DataFrame(squareform(pdist(similar_times.iloc[:, 1:])), columns=similar_times.index.unique(), index=similar_times.index.unique())
    # Remove query runner from list of neighbours
    euclidean_distance = euclidean_distance.drop([index], axis = 0)
    euclidean_distance = euclidean_distance.loc[:,index].sort_values(ascending = True)
    #euclidean_distance = euclidean_distance[euclidean_distance < euclidean_distance_threshold]
    #euclidean_distance = euclidean_distance.head(10)
   # euclidean_distance_1 = euclidean_distance.head(1)
    #euclidean_distance_2 = euclidean_distance.head(2)
    euclidean_distance = euclidean_distance.head(5)
    #euclidean_distance_20 = euclidean_distance.head(20)
    return euclidean_distance

# Show sample output
calculate_euclidean_distance(0)

In [ ]:
def find_nearest_neighbours(index):
    euclidean_distance = calculate_euclidean_distance(index)  
    #should this be non pb time???????
    nearest_neighbours = merge.iloc[euclidean_distance.index]
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
    return nearest_neighbours
    
# Show sample output    
find_nearest_neighbours(0).head()

In [ ]:
def get_mean_pb(neighbours):
    return neighbours.AVG_Pace_pb.mean()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
get_mean_pb(neighbours)

In [ ]:
def get_mean_pace(neighbours):
    return neighbours.mean()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
neighbours = neighbours.filter(like='pb')
get_mean_pace(neighbours)

In [ ]:
def get_best_npb(neighbours):
    return neighbours.AVG_Pace.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
get_best_npb(neighbours)

In [ ]:
def get_best_pb(neighbours):
    return neighbours.AVG_Pace_pb.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
get_best_pb(neighbours)

In [ ]:
def Cham_to_Del(neighbours):
    return neighbours['Chamonix to Delevret_pb'].mean()

def Cham_to_LaCh(neighbours):
    return neighbours['Chamonix to La Charme_pb'].mean()

def Cham_to_StG(neighbours):
    return neighbours['Chamonix to St-Gervais_pb'].mean()

def Del_to_StG(neighbours):
    return neighbours['Delevret to St-Gervais_pb'].mean()

def LaCh_to_StG(neighbours):
    return neighbours['La Charme to St-Gervais_pb'].mean()

def StG_to_Cont(neighbours):
    return neighbours['St-Gervais to Contamines_pb'].mean()

def Cont_to_LaB(neighbours):
    return neighbours['Contamines to La Balme_pb'].mean()

def LaB_to_Bon(neighbours):
    return neighbours['La Balme to Bonhomme_pb'].mean()

def Bon_to_Chap(neighbours):
    return neighbours['Bonhomme to Chapieux_pb'].mean()

def Chap_to_ColS(neighbours):
    return neighbours['Chapieux to Col Seigne_pb'].mean()

def ColS_to_Lac(neighbours):
    return neighbours['Col Seigne to Lac Combal_pb'].mean()

def Lac_to_MtF(neighbours):
    return neighbours['Lac Combal to Mt-Favre_pb'].mean()

def MtF_to_Chec(neighbours):
    return neighbours['Mt-Favre to Checruit_pb'].mean()

def Chec_to_Courm(neighbours):
    return neighbours['Checruit to Courmayeur_pb'].mean()

def Courm_to_Courm2(neighbours):
    return neighbours['Courmayeur to Courmayeur2_pb'].mean()

def Courm2_to_Bertone(neighbours):
    return neighbours['Courmayeur2 to Bertone_pb'].mean()

def Courm_to_Bertone(neighbours):
    return neighbours['Courmayeur to Bertone_pb'].mean()

def Bertone_to_Bon(neighbours):
    return neighbours['Bertone to Bonatti_pb'].mean()

def Bon_to_Arn(neighbours):
    return neighbours['Bonatti to Arnouvaz_pb'].mean()

def Arn_to_Col(neighbours):
    return neighbours['Arnouvaz to Col Ferret_pb'].mean()

def Col_to_LaF(neighbours):
    return neighbours['Col Ferret to La Fouly_pb'].mean()

def LaF_to_Champ(neighbours):
    return neighbours['La Fouly to Champex La_pb'].mean()

def Champ_to_Bov(neighbours):
    return neighbours['Champex La to Bovine_pb'].mean()

def Champ_to_Mort(neighbours):
    return neighbours['Champex La to Mortigny_pb'].mean()

def Champ_to_Giete(neighbours):
    return neighbours['Champex La to Giete_pb'].mean()

def Bov_to_Trient(neighbours):
    return neighbours['Bovine to Trient_pb'].mean()

def Mort_to_Trient(neighbours):
    return neighbours['Mortigny to Trient_pb'].mean()

def Giete_to_Trient(neighbours):
    return neighbours['Giete to Trient_pb'].mean()

def Trient_to_LaCa(neighbours):
    return neighbours['Trient to La Catogne_pb'].mean()

def Trient_to_LesSt(neighbours):
    return neighbours['Trient to Les Tseppe_pb'].mean()

def LaCa_to_Vall(neighbours):
    return neighbours['La Catogne to Vallorcine_pb'].mean()

def LesSt_to_Vall(neighbours):
    return neighbours['Les Tseppe to Vallorcine_pb'].mean()

def Vall_to_tAV(neighbours):
    return neighbours['Vallorcine to Tete aux Vents_pb'].mean()

def Vall_to_Arg(neighbours):
    return neighbours['Vallorcine to Argentiere_pb'].mean()

def Vall_to_Col(neighbours):
    return neighbours['Vallorcine to Col Montet_pb'].mean()

def tAV_to_Fleg(neighbours):
    return neighbours['Tete aux Vents to Flegere_pb'].mean()

def Arg_to_Cham(neighbours):
    return neighbours['Argentiere to Chamonix_pb'].mean()

def Col_to_Fleg(neighbours):
    return neighbours['Col Montet to Flegere_pb'].mean()

def Fleg_to_Cham(neighbours):
    return neighbours['Flegere to Chamonix_pb'].mean()

# Show sample output    
neighbours = find_nearest_neighbours(0)
#neighbours = neighbours.filter(like='pb')
#get_best_pace(neighbours)
Cham_to_StG(neighbours)
#Cham_to_LaCh(neighbours)
#Cham_to_StG(neighbours)   


In [ ]:
def get_prediction(index, neighbours):
    weight = merge.iloc[index].AVG_Pace/get_best_npb(neighbours)
    best_pb = get_best_pb(neighbours)
    return best_pb * weight

# Show output
neighbours = find_nearest_neighbours(0)
get_prediction(0, neighbours)

## Calculate Predictions

In [ ]:
def update_rows(row):
    index = int(row['best_pb_prediction'])
    row['query_AVG_Pace'] = merge.iloc[index].AVG_Pace
    row['query_pb'] = merge.iloc[index].AVG_Pace_pb
    neighbours = find_nearest_neighbours(index)
    row['mean_pb_average_pace'] = get_mean_pb(neighbours)
  #  row['best_average_pace'] = get_best_npb(neighbours)
    row['best_pb_prediction'] = get_prediction(index, neighbours)
    return row

In [ ]:
num_of_runs = merge.time.count()
filler_data = np.arange(0.0, num_of_runs)
df_k5 = pd.DataFrame({'best_pb_prediction':filler_data, 'mean_pb_average_pace':filler_data, 'query_AVG_Pace':filler_data, 'query_pb':filler_data})

df_k5 = df_k5.apply(lambda row: update_rows(row), axis=1)
df_k5 = df_k5[['query_AVG_Pace', 'query_pb', 'best_pb_prediction', 'mean_pb_average_pace']]
df_k5

In [ ]:
df_k5 = comb.join(df_k5)

In [ ]:
df_k5.to_csv('../../data/K_5_pace_predictions.csv')

# K = 20

## Define Functions

In [ ]:
def find_similar_times(index):
    query = npb.iloc[index]
    same_category = npb[npb.category == query.category]
    similar_times = same_category[(same_category.AVG_Pace > query.AVG_Pace - pace_threshold) & (same_category.AVG_Pace < query.AVG_Pace + pace_threshold)]
    return similar_times

# Show sample output
find_similar_times(0)

In [ ]:
def calculate_euclidean_distance(index):
    similar_times = find_similar_times(index)
    similar_times = similar_times.drop(['category', 'name_mask', 'Year'], axis=1)
    similar_times = similar_times.fillna(0)
    euclidean_distance = pd.DataFrame(squareform(pdist(similar_times.iloc[:, 1:])), columns=similar_times.index.unique(), index=similar_times.index.unique())
    # Remove query runner from list of neighbours
    euclidean_distance = euclidean_distance.drop([index], axis = 0)
    euclidean_distance = euclidean_distance.loc[:,index].sort_values(ascending = True)
    #euclidean_distance = euclidean_distance[euclidean_distance < euclidean_distance_threshold]
    #euclidean_distance = euclidean_distance.head(10)
    #euclidean_distance_1 = euclidean_distance.head(1)
    #euclidean_distance_2 = euclidean_distance.head(2)
    #euclidean_distance_5 = euclidean_distance.head(5)
    euclidean_distance = euclidean_distance.head(20)
    return euclidean_distance

# Show sample output
calculate_euclidean_distance(0)

In [ ]:
def find_nearest_neighbours(index):
    euclidean_distance = calculate_euclidean_distance(index)  
    #should this be non pb time???????
    nearest_neighbours = merge.iloc[euclidean_distance.index]
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
    return nearest_neighbours
    
# Show sample output    
find_nearest_neighbours(0).head()

In [ ]:
def get_mean_pb(neighbours):
    return neighbours.AVG_Pace_pb.mean()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
get_mean_pb(neighbours)

In [ ]:
def get_mean_pace(neighbours):
    return neighbours.mean()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
neighbours = neighbours.filter(like='pb')
get_mean_pace(neighbours)

In [ ]:
def get_best_npb(neighbours):
    return neighbours.AVG_Pace.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
get_best_npb(neighbours)

In [ ]:
def get_best_pb(neighbours):
    return neighbours.AVG_Pace_pb.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
get_best_pb(neighbours)

In [ ]:
def Cham_to_Del(neighbours):
    return neighbours['Chamonix to Delevret_pb'].mean()

def Cham_to_LaCh(neighbours):
    return neighbours['Chamonix to La Charme_pb'].mean()

def Cham_to_StG(neighbours):
    return neighbours['Chamonix to St-Gervais_pb'].mean()

def Del_to_StG(neighbours):
    return neighbours['Delevret to St-Gervais_pb'].mean()

def LaCh_to_StG(neighbours):
    return neighbours['La Charme to St-Gervais_pb'].mean()

def StG_to_Cont(neighbours):
    return neighbours['St-Gervais to Contamines_pb'].mean()

def Cont_to_LaB(neighbours):
    return neighbours['Contamines to La Balme_pb'].mean()

def LaB_to_Bon(neighbours):
    return neighbours['La Balme to Bonhomme_pb'].mean()

def Bon_to_Chap(neighbours):
    return neighbours['Bonhomme to Chapieux_pb'].mean()

def Chap_to_ColS(neighbours):
    return neighbours['Chapieux to Col Seigne_pb'].mean()

def ColS_to_Lac(neighbours):
    return neighbours['Col Seigne to Lac Combal_pb'].mean()

def Lac_to_MtF(neighbours):
    return neighbours['Lac Combal to Mt-Favre_pb'].mean()

def MtF_to_Chec(neighbours):
    return neighbours['Mt-Favre to Checruit_pb'].mean()

def Chec_to_Courm(neighbours):
    return neighbours['Checruit to Courmayeur_pb'].mean()

def Courm_to_Courm2(neighbours):
    return neighbours['Courmayeur to Courmayeur2_pb'].mean()

def Courm2_to_Bertone(neighbours):
    return neighbours['Courmayeur2 to Bertone_pb'].mean()

def Courm_to_Bertone(neighbours):
    return neighbours['Courmayeur to Bertone_pb'].mean()

def Bertone_to_Bon(neighbours):
    return neighbours['Bertone to Bonatti_pb'].mean()

def Bon_to_Arn(neighbours):
    return neighbours['Bonatti to Arnouvaz_pb'].mean()

def Arn_to_Col(neighbours):
    return neighbours['Arnouvaz to Col Ferret_pb'].mean()

def Col_to_LaF(neighbours):
    return neighbours['Col Ferret to La Fouly_pb'].mean()

def LaF_to_Champ(neighbours):
    return neighbours['La Fouly to Champex La_pb'].mean()

def Champ_to_Bov(neighbours):
    return neighbours['Champex La to Bovine_pb'].mean()

def Champ_to_Mort(neighbours):
    return neighbours['Champex La to Mortigny_pb'].mean()

def Champ_to_Giete(neighbours):
    return neighbours['Champex La to Giete_pb'].mean()

def Bov_to_Trient(neighbours):
    return neighbours['Bovine to Trient_pb'].mean()

def Mort_to_Trient(neighbours):
    return neighbours['Mortigny to Trient_pb'].mean()

def Giete_to_Trient(neighbours):
    return neighbours['Giete to Trient_pb'].mean()

def Trient_to_LaCa(neighbours):
    return neighbours['Trient to La Catogne_pb'].mean()

def Trient_to_LesSt(neighbours):
    return neighbours['Trient to Les Tseppe_pb'].mean()

def LaCa_to_Vall(neighbours):
    return neighbours['La Catogne to Vallorcine_pb'].mean()

def LesSt_to_Vall(neighbours):
    return neighbours['Les Tseppe to Vallorcine_pb'].mean()

def Vall_to_tAV(neighbours):
    return neighbours['Vallorcine to Tete aux Vents_pb'].mean()

def Vall_to_Arg(neighbours):
    return neighbours['Vallorcine to Argentiere_pb'].mean()

def Vall_to_Col(neighbours):
    return neighbours['Vallorcine to Col Montet_pb'].mean()

def tAV_to_Fleg(neighbours):
    return neighbours['Tete aux Vents to Flegere_pb'].mean()

def Arg_to_Cham(neighbours):
    return neighbours['Argentiere to Chamonix_pb'].mean()

def Col_to_Fleg(neighbours):
    return neighbours['Col Montet to Flegere_pb'].mean()

def Fleg_to_Cham(neighbours):
    return neighbours['Flegere to Chamonix_pb'].mean()

# Show sample output    
neighbours = find_nearest_neighbours(0)
#neighbours = neighbours.filter(like='pb')
#get_best_pace(neighbours)
Cham_to_StG(neighbours)
#Cham_to_LaCh(neighbours)
#Cham_to_StG(neighbours)   


In [ ]:
def get_prediction(index, neighbours):
    weight = merge.iloc[index].AVG_Pace/get_best_npb(neighbours)
    best_pb = get_best_pb(neighbours)
    return best_pb * weight

# Show output
neighbours = find_nearest_neighbours(0)
get_prediction(0, neighbours)

## Calculate Predictions

In [ ]:
def update_rows(row):
    index = int(row['best_pb_prediction'])
    row['query_AVG_Pace'] = merge.iloc[index].AVG_Pace
    row['query_pb'] = merge.iloc[index].AVG_Pace_pb
    neighbours = find_nearest_neighbours(index)
    row['mean_pb_average_pace'] = get_mean_pb(neighbours)
  #  row['best_average_pace'] = get_best_npb(neighbours)
    row['best_pb_prediction'] = get_prediction(index, neighbours)
    return row

In [ ]:
num_of_runs = merge.time.count()
filler_data = np.arange(0.0, num_of_runs)
df_k20 = pd.DataFrame({'best_pb_prediction':filler_data, 'mean_pb_average_pace':filler_data, 'query_AVG_Pace':filler_data, 'query_pb':filler_data})

df_k20 = df_k20.apply(lambda row: update_rows(row), axis=1)
df_k20 = df_k20[['query_AVG_Pace', 'query_pb', 'best_pb_prediction', 'mean_pb_average_pace']]
df_k20

In [ ]:
df_k20 = comb.join(df_k20)

In [ ]:
df_k20.to_csv('../../data/K_20_pace_predictions.csv')

# K = 1 

## Define Functions

In [ ]:
def find_similar_times(index):
    query = npb.iloc[index]
    same_category = npb[npb.category == query.category]
    similar_times = same_category[(same_category.AVG_Pace > query.AVG_Pace - pace_threshold) & (same_category.AVG_Pace < query.AVG_Pace + pace_threshold)]
    return similar_times

# Show sample output
find_similar_times(0)

In [ ]:
def calculate_euclidean_distance(index):
    similar_times = find_similar_times(index)
    similar_times = similar_times.drop(['category', 'name_mask', 'Year'], axis=1)
    similar_times = similar_times.fillna(0)
    euclidean_distance = pd.DataFrame(squareform(pdist(similar_times.iloc[:, 1:])), columns=similar_times.index.unique(), index=similar_times.index.unique())
    # Remove query runner from list of neighbours
    euclidean_distance = euclidean_distance.drop([index], axis = 0)
    euclidean_distance = euclidean_distance.loc[:,index].sort_values(ascending = True)
    #euclidean_distance = euclidean_distance[euclidean_distance < euclidean_distance_threshold]
    #euclidean_distance = euclidean_distance.head(10)
    euclidean_distance = euclidean_distance.head(1)
    #euclidean_distance_2 = euclidean_distance.head(2)
    #euclidean_distance_5 = euclidean_distance.head(5)
    #euclidean_distance_20 = euclidean_distance.head(20)
    return euclidean_distance

# Show sample output
calculate_euclidean_distance(0)

In [ ]:
def find_nearest_neighbours(index):
    euclidean_distance = calculate_euclidean_distance(index)  
    #should this be non pb time???????
    nearest_neighbours = merge.iloc[euclidean_distance.index]
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
    return nearest_neighbours
    
# Show sample output    
find_nearest_neighbours(0).head()

In [ ]:
def get_mean_pb(neighbours):
    return neighbours.AVG_Pace_pb.mean()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
get_mean_pb(neighbours)

In [ ]:
def get_mean_pace(neighbours):
    return neighbours.mean()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
neighbours = neighbours.filter(like='pb')
get_mean_pace(neighbours)

In [ ]:
def get_best_npb(neighbours):
    return neighbours.AVG_Pace.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
get_best_npb(neighbours)

In [ ]:
def get_best_pb(neighbours):
    return neighbours.AVG_Pace_pb.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
get_best_pb(neighbours)

In [ ]:
def Cham_to_Del(neighbours):
    return neighbours['Chamonix to Delevret_pb'].mean()

def Cham_to_LaCh(neighbours):
    return neighbours['Chamonix to La Charme_pb'].mean()

def Cham_to_StG(neighbours):
    return neighbours['Chamonix to St-Gervais_pb'].mean()

def Del_to_StG(neighbours):
    return neighbours['Delevret to St-Gervais_pb'].mean()

def LaCh_to_StG(neighbours):
    return neighbours['La Charme to St-Gervais_pb'].mean()

def StG_to_Cont(neighbours):
    return neighbours['St-Gervais to Contamines_pb'].mean()

def Cont_to_LaB(neighbours):
    return neighbours['Contamines to La Balme_pb'].mean()

def LaB_to_Bon(neighbours):
    return neighbours['La Balme to Bonhomme_pb'].mean()

def Bon_to_Chap(neighbours):
    return neighbours['Bonhomme to Chapieux_pb'].mean()

def Chap_to_ColS(neighbours):
    return neighbours['Chapieux to Col Seigne_pb'].mean()

def ColS_to_Lac(neighbours):
    return neighbours['Col Seigne to Lac Combal_pb'].mean()

def Lac_to_MtF(neighbours):
    return neighbours['Lac Combal to Mt-Favre_pb'].mean()

def MtF_to_Chec(neighbours):
    return neighbours['Mt-Favre to Checruit_pb'].mean()

def Chec_to_Courm(neighbours):
    return neighbours['Checruit to Courmayeur_pb'].mean()

def Courm_to_Courm2(neighbours):
    return neighbours['Courmayeur to Courmayeur2_pb'].mean()

def Courm2_to_Bertone(neighbours):
    return neighbours['Courmayeur2 to Bertone_pb'].mean()

def Courm_to_Bertone(neighbours):
    return neighbours['Courmayeur to Bertone_pb'].mean()

def Bertone_to_Bon(neighbours):
    return neighbours['Bertone to Bonatti_pb'].mean()

def Bon_to_Arn(neighbours):
    return neighbours['Bonatti to Arnouvaz_pb'].mean()

def Arn_to_Col(neighbours):
    return neighbours['Arnouvaz to Col Ferret_pb'].mean()

def Col_to_LaF(neighbours):
    return neighbours['Col Ferret to La Fouly_pb'].mean()

def LaF_to_Champ(neighbours):
    return neighbours['La Fouly to Champex La_pb'].mean()

def Champ_to_Bov(neighbours):
    return neighbours['Champex La to Bovine_pb'].mean()

def Champ_to_Mort(neighbours):
    return neighbours['Champex La to Mortigny_pb'].mean()

def Champ_to_Giete(neighbours):
    return neighbours['Champex La to Giete_pb'].mean()

def Bov_to_Trient(neighbours):
    return neighbours['Bovine to Trient_pb'].mean()

def Mort_to_Trient(neighbours):
    return neighbours['Mortigny to Trient_pb'].mean()

def Giete_to_Trient(neighbours):
    return neighbours['Giete to Trient_pb'].mean()

def Trient_to_LaCa(neighbours):
    return neighbours['Trient to La Catogne_pb'].mean()

def Trient_to_LesSt(neighbours):
    return neighbours['Trient to Les Tseppe_pb'].mean()

def LaCa_to_Vall(neighbours):
    return neighbours['La Catogne to Vallorcine_pb'].mean()

def LesSt_to_Vall(neighbours):
    return neighbours['Les Tseppe to Vallorcine_pb'].mean()

def Vall_to_tAV(neighbours):
    return neighbours['Vallorcine to Tete aux Vents_pb'].mean()

def Vall_to_Arg(neighbours):
    return neighbours['Vallorcine to Argentiere_pb'].mean()

def Vall_to_Col(neighbours):
    return neighbours['Vallorcine to Col Montet_pb'].mean()

def tAV_to_Fleg(neighbours):
    return neighbours['Tete aux Vents to Flegere_pb'].mean()

def Arg_to_Cham(neighbours):
    return neighbours['Argentiere to Chamonix_pb'].mean()

def Col_to_Fleg(neighbours):
    return neighbours['Col Montet to Flegere_pb'].mean()

def Fleg_to_Cham(neighbours):
    return neighbours['Flegere to Chamonix_pb'].mean()

# Show sample output    
neighbours = find_nearest_neighbours(0)
#neighbours = neighbours.filter(like='pb')
#get_best_pace(neighbours)
Cham_to_StG(neighbours)
#Cham_to_LaCh(neighbours)
#Cham_to_StG(neighbours)   


In [ ]:
def get_prediction(index, neighbours):
    weight = merge.iloc[index].AVG_Pace/get_best_npb(neighbours)
    best_pb = get_best_pb(neighbours)
    return best_pb * weight

# Show output
neighbours = find_nearest_neighbours(0)
get_prediction(0, neighbours)

## Calculate Predictions

In [ ]:
def update_rows(row):
    index = int(row['best_pb_prediction'])
    row['query_AVG_Pace'] = merge.iloc[index].AVG_Pace
    row['query_pb'] = merge.iloc[index].AVG_Pace_pb
    neighbours = find_nearest_neighbours(index)
    row['mean_pb_average_pace'] = get_mean_pb(neighbours)
  #  row['best_average_pace'] = get_best_npb(neighbours)
    row['best_pb_prediction'] = get_prediction(index, neighbours)
    return row

In [ ]:
num_of_runs = merge.time.count()
filler_data = np.arange(0.0, num_of_runs)
df_k1 = pd.DataFrame({'best_pb_prediction':filler_data, 'mean_pb_average_pace':filler_data, 'query_AVG_Pace':filler_data, 'query_pb':filler_data})

df_k1 = df_k1.apply(lambda row: update_rows(row), axis=1)
df_k1 = df_k1[['query_AVG_Pace', 'query_pb', 'best_pb_prediction', 'mean_pb_average_pace']]
df_k1

In [ ]:
df_k1 = comb.join(df_k1)

In [ ]:
df_k1.to_csv('../../data/K_1_pace_predictions.csv')

# K = 3

## Define Functions

In [ ]:
def find_similar_times(index):
    query = npb.iloc[index]
    same_category = npb[npb.category == query.category]
    similar_times = same_category[(same_category.AVG_Pace > query.AVG_Pace - pace_threshold) & (same_category.AVG_Pace < query.AVG_Pace + pace_threshold)]
    return similar_times

# Show sample output
find_similar_times(0)

In [ ]:
def calculate_euclidean_distance(index):
    similar_times = find_similar_times(index)
    similar_times = similar_times.drop(['category', 'name_mask', 'Year'], axis=1)
    similar_times = similar_times.fillna(0)
    euclidean_distance = pd.DataFrame(squareform(pdist(similar_times.iloc[:, 1:])), columns=similar_times.index.unique(), index=similar_times.index.unique())
    # Remove query runner from list of neighbours
    euclidean_distance = euclidean_distance.drop([index], axis = 0)
    euclidean_distance = euclidean_distance.loc[:,index].sort_values(ascending = True)
    #euclidean_distance = euclidean_distance[euclidean_distance < euclidean_distance_threshold]
    #euclidean_distance = euclidean_distance.head(10)
    euclidean_distance = euclidean_distance.head(3)
    #euclidean_distance_2 = euclidean_distance.head(2)
    #euclidean_distance_5 = euclidean_distance.head(5)
    #euclidean_distance_20 = euclidean_distance.head(20)
    return euclidean_distance

# Show sample output
calculate_euclidean_distance(0)

In [ ]:
def find_nearest_neighbours(index):
    euclidean_distance = calculate_euclidean_distance(index)  
    #should this be non pb time???????
    nearest_neighbours = merge.iloc[euclidean_distance.index]
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
    return nearest_neighbours
    
# Show sample output    
find_nearest_neighbours(0).head()

In [ ]:
def get_mean_pb(neighbours):
    return neighbours.AVG_Pace_pb.mean()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
get_mean_pb(neighbours)

In [ ]:
def get_mean_pace(neighbours):
    return neighbours.mean()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
neighbours = neighbours.filter(like='pb')
get_mean_pace(neighbours)

In [ ]:
def get_best_npb(neighbours):
    return neighbours.AVG_Pace.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
get_best_npb(neighbours)

In [ ]:
def get_best_pb(neighbours):
    return neighbours.AVG_Pace_pb.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(0)
get_best_pb(neighbours)

In [ ]:
def get_prediction(index, neighbours):
    weight = merge.iloc[index].AVG_Pace/get_best_npb(neighbours)
    best_pb = get_best_pb(neighbours)
    return best_pb * weight

# Show output
neighbours = find_nearest_neighbours(0)
get_prediction(0, neighbours)

## Calculate Predictions

In [ ]:
def update_rows(row):
    index = int(row['best_pb_prediction'])
    row['query_AVG_Pace'] = merge.iloc[index].AVG_Pace
    row['query_pb'] = merge.iloc[index].AVG_Pace_pb
    neighbours = find_nearest_neighbours(index)
    row['mean_pb_average_pace'] = get_mean_pb(neighbours)
  #  row['best_average_pace'] = get_best_npb(neighbours)
    row['best_pb_prediction'] = get_prediction(index, neighbours)
    return row

In [ ]:
num_of_runs = merge.time.count()
filler_data = np.arange(0.0, num_of_runs)
df_k3 = pd.DataFrame({'best_pb_prediction':filler_data, 'mean_pb_average_pace':filler_data, 'query_AVG_Pace':filler_data, 'query_pb':filler_data})

df_k3 = df_k3.apply(lambda row: update_rows(row), axis=1)
df_k3 = df_k3[['query_AVG_Pace', 'query_pb', 'best_pb_prediction', 'mean_pb_average_pace']]
df_k3

In [ ]:
df_k3 = comb.join(df_k3)

In [ ]:
df_k3.to_csv('../../data/K_3_pace_predictions.csv')